In [47]:
from ase import Atoms
from ase.io import read
import xcquinox as xce
import torch, jax, optax
import numpy as np
import equinox as eqx
import jax.numpy as jnp
import pyscfad as psa
import os, sys
from pyscfad import dft, scf, gto, df
from pyscfad.pbc import scf as scfp
from pyscfad.pbc import gto as gtop
from pyscfad.pbc import dft as dftp
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'

from mp_api.client import MPRester

In [49]:
mpr = MPRester(api_key = 'UutJcvAlT6PETlthXKpvFNDU3VjztR1c')

In [146]:
ret = mpr.get_bandstructure_by_material_id('mp-66')

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [147]:
struc = ret.structure

In [156]:
struc.get_reduced_structure()

Structure Summary
Lattice
    abc : 2.526994574994177 2.526994574994177 2.526994574994177
 angles : 60.00000000000001 60.00000000000001 60.00000000000001
 volume : 11.410322800393955
      A : -1.786855 -1.786855 0.0
      B : -1.786855 0.0 -1.786855
      C : 0.0 -1.786855 -1.786855
    pbc : True True True
PeriodicSite: C (-2.68, -2.68, -2.68) [0.75, 0.75, 0.75]
PeriodicSite: C (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

In [149]:
rets = ret.structure.as_dict()
at_coor_xyz = [ (i['species'][0]['element'], i['xyz']) for i in rets['sites']]

at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]

In [151]:
at_coor_abc

[('C', [1.8952459312456327, 1.8952459312456327, 1.8952459312456327]),
 ('C', [0.0, 0.0, 0.0])]

In [153]:
at_coor_xyz

[('C', [-2.6802825, -2.6802825, -2.6802825]), ('C', [0.0, 0.0, 0.0])]

In [115]:
cella = -np.asarray(rets['lattice']['matrix'])

In [154]:
tcell = gtop.Cell()
cell.atom = at_coor_xyz
cell.a = cella
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build(trace_lattice_vectors=True)
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()

LinAlgError: The leading minor of order 2 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.

In [98]:
rets['sites'][0]

{'species': [{'element': 'Si', 'occu': 1}],
 'abc': [0.75, 0.75, 0.75],
 'xyz': [-4.1016945, -4.1016945, -4.1016945],
 'properties': {},
 'label': 'Si'}

In [76]:
ret.get_vbm()

{'band_index': defaultdict(list, {<Spin.up: 1>: [1, 2, 3]}),
 'kpoint_index': [0, 72, 73],
 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint at 0x748e25c9f8e0>,
 'energy': 5.6165,
 'projections': {<Spin.up: 1>: array([[0.    , 0.    ],
         [0.0066, 0.0066],
         [0.1974, 0.1974],
         [0.0098, 0.0098],
         [0.    , 0.    ],
         [0.    , 0.    ],
         [0.    , 0.    ],
         [0.    , 0.    ],
         [0.    , 0.    ]])}}

In [67]:
ret.get_band_gap??

Signature: ret.get_band_gap()
Source:   
    def get_band_gap(self):
        r"""Returns band gap data.

        Returns:
            A dict {"energy","direct","transition"}:
            "energy": band gap energy
            "direct": A boolean telling if the gap is direct or not
            "transition": kpoint labels of the transition (e.g., "\\Gamma-X")
        """
        if self.is_metal():
            return {"energy": 0.0, "direct": False, "transition": None}
        cbm = self.get_cbm()
        vbm = self.get_vbm()
        result = {"direct": False, "energy": 0.0, "transition": None}

        result["energy"] = cbm["energy"] - vbm["energy"]

        if (cbm["kpoint"].label is not None and cbm["kpoint"].label == vbm["kpoint"].label) or np.linalg.norm(
            cbm["kpoint"].cart_coords - vbm["kpoint"].cart_coords
        ) < 0.01:
            result["direct"] = True

        result["transition"] = "-".join(
            [
                str(c.label) if c.label is not None els

Non-local descriptor package

In [2]:
from mldftdat.density import *
from mldftdat.density import _get_x_helper_c
from mldftdat.pyscf_utils import get_mgga_data
from mldftdat.analyzers import *

Utility function requiring torch to load old models, but we won't require torch as a prerequisite. Eventually want to have a self-contained folder of translated models.

Torch structure for loading old models below.

In [3]:
# #relevant functions from dpyscfl to see if it can be self-contained here in the notebook
# #xcdiff has this named XC_L, not X_L. keep for consistency's sake

# class LOB(torch.nn.Module):

#     def __init__(self, limit=1.804):
#         """ Utility function to squash output to [-1, limit-1] inteval.
#             Can be used to enforce non-negativity and Lieb-Oxford bound.
#         """
#         super().__init__()
#         self.sig = torch.nn.Sigmoid()
#         self.limit = limit

#     def forward(self, x):
#         return self.limit*self.sig(x-np.log(self.limit-1))-1


# class X_L(torch.nn.Module):
#     def __init__(self, n_input, n_hidden=16, use=[], device='cpu', ueg_limit=False, lob=1.804, one_e=False):
#         """Local exchange model based on MLP
#         Receives density descriptors in this order : [rho, s, alpha, nl],
#         input may be truncated depending on level of approximation

#         Args:
#             n_input (int): Input dimensions (LDA: 1, GGA: 2, meta-GGA: 3, ...)
#             n_hidden (int, optional): Number of hidden nodes (three hidden layers used by default). Defaults to 16.
#             use (list of ints, optional): Only these indices are used as input to the model (can be used to omit density as input to enforce uniform density scaling). These indices are also used to enforce UEG where the assumed order is [s, alpha, ...].. Defaults to [].
#             device (str, optional): {'cpu','cuda'}. Defaults to 'cpu'.
#             ueg_limit (bool, optional): Enforce uniform homoegeneous electron gas limit. Defaults to False.
#             lob (float, optional): Enforce this value as local Lieb-Oxford bound (don't enforce if set to 0). Defaults to 1.804.
#             one_e (bool, optional): _description_. Defaults to False.
#         """
#         super().__init__()
#         self.ueg_limit = ueg_limit
#         self.spin_scaling = True
#         self.lob = lob

#         if not use:
#             self.use = torch.Tensor(np.arange(n_input)).long().to(device)
#         else:
#             self.use = torch.Tensor(use).long().to(device)
#         #xcdiff includes double flag on net
#         self.net =  torch.nn.Sequential(
#                 torch.nn.Linear(n_input, n_hidden),
#                 torch.nn.GELU(),
#                 torch.nn.Linear(n_hidden, n_hidden),
#                 torch.nn.GELU(),
#                 torch.nn.Linear(n_hidden, n_hidden),
#                 torch.nn.GELU(),
#                 torch.nn.Linear(n_hidden, 1),
#             ).double().to(device)

#         #to device not declared in xcdiff
#         self.tanh = torch.nn.Tanh().to(device)
#         self.lobf = LOB(lob).to(device)
#         #below declared in xcdiff
#         self.sig = torch.nn.Sigmoid()
#         self.shift = 1/(1+np.exp(-1e-3))

#     def forward(self, rho, **kwargs):
#         """Forward pass

#         Args:
#             rho (_type_): _description_

#         Returns:
#             _type_: _description_
#         """
#         # print(rho.size, rho.shape, rho.dtype)
#         # print('x call -- rho shape', rho.shape)
#         # print('x call -- rho[...,self.use] shape', rho[...,self.use].shape)
#         squeezed = self.net(rho[...,self.use]).squeeze()
#         # print('x call -- squeezed shape', squeezed.shape)
#         # print('x call -- squeezed', squeezed)

#         if self.ueg_limit:
#             ueg_lim = rho[...,self.use[0]]
#             if len(self.use) > 1:
#                 ueg_lim_a = torch.pow(self.tanh(rho[...,self.use[1]]),2)
#             else:
#                 ueg_lim_a = 0
#             #below comparison not in xcdiff
#             if len(self.use) > 2:
#                 ueg_lim_nl = torch.sum(rho[...,self.use[2:]],dim=-1)
#             else:
#                 ueg_lim_nl = 0
#         else:
#             ueg_lim = 1
#             ueg_lim_a = 0
#             ueg_lim_nl = 0

#         if self.lob:
#             result = self.lobf(squeezed*(ueg_lim + ueg_lim_a + ueg_lim_nl))
#         else:
#             result = squeezed*(ueg_lim + ueg_lim_a + ueg_lim_nl)

#         return result

# class C_L(torch.nn.Module):
#     def __init__(self, n_input=2,n_hidden=16, device='cpu', ueg_limit=False, lob=2.0, use = []):
#         """Local correlation model based on MLP
#         Receives density descriptors in this order : [rho, spinscale, s, alpha, nl]
#         input may be truncated depending on level of approximation

#         Args:
#             n_input (int, optional): Input dimensions (LDA: 2, GGA: 3 , meta-GGA: 4). Defaults to 2.
#             n_hidden (int, optional): Number of hidden nodes (three hidden layers used by default). Defaults to 16.
#             device (str, optional): {'cpu','cuda'}. Defaults to 'cpu'.
#             ueg_limit (bool, optional): Enforce uniform homoegeneous electron gas limit. Defaults to False.
#             lob (float, optional): Technically Lieb-Oxford bound but used here to enforce non-negativity. Should be kept at 2.0 in most instances. Defaults to 2.0.
#             use (list of ints, optional): Indices for [s, alpha] (in that order) in input, to determine UEG limit. Defaults to [].
#         """
#         super().__init__()
#         self.spin_scaling = False
#         self.lob = False
#         self.ueg_limit = ueg_limit
#         self.n_input=n_input

#         if not use:
#             self.use = torch.Tensor(np.arange(n_input)).long().to(device)
#         else:
#             self.use = torch.Tensor(use).long().to(device)
#         self.net = torch.nn.Sequential(
#                 torch.nn.Linear(n_input, n_hidden),
#                 torch.nn.GELU(),
#                 torch.nn.Linear(n_hidden, n_hidden),
#                 torch.nn.GELU(),
#                 torch.nn.Linear(n_hidden, n_hidden),
#                 torch.nn.GELU(),
#                 torch.nn.Linear(n_hidden, 1),
#                 torch.nn.Softplus()
#             ).double().to(device)
#         self.sig = torch.nn.Sigmoid()

#         self.tanh = torch.nn.Tanh()
#         #self.lob section allows for different values here, default=2. xcdiff doesn't have this,
#         #assumes 2 always
#         self.lob = lob
#         if self.lob:
#             self.lobf = LOB(self.lob)
#         else:
#             self.lob =  1000.0
#             self.lobf = LOB(self.lob)


#     def forward(self, rho, **kwargs):
#         """Forward pass in network

#         Args:
#             rho (torch.Tensor): density

#         Returns:
#             _type_: _description_
#         """
#         inp = rho
#         # print(rho.size, rho.shape, rho.dtype)
#         # print('c call -- rho shape', rho.shape)
#         # print('c call, rho[...,self.use] shape', rho.shape)
#         # print('c call, rho[...,self.use]', rho)        
#         squeezed = -self.net(inp).squeeze()
#         # print('c call -- squeezed shape', squeezed.shape)
#         # print('c call -- squeezed', squeezed)
        
#         if self.ueg_limit:
#             #below not form used in xcdiff
# #            ueg_lim = rho[...,self.use[0]]
#             #below form used in xcdiff,
#             ueg_lim = self.tanh(rho[...,self.use[0]])
#             if len(self.use) > 1:
#                 ueg_lim_a = torch.pow(self.tanh(rho[...,self.use[1]]),2)
#             else:
#                 ueg_lim_a = 0
#             #xcdiff does not include this next comparison
#             if len(self.use) > 2:
#                 ueg_lim_nl = torch.sum(self.tanh(rho[...,self.use[2:]])**2,dim=-1)
#             else:
#                 ueg_lim_nl = 0

#             ueg_factor = ueg_lim + ueg_lim_a + ueg_lim_nl
#         else:
#             ueg_factor = 1
#         #xcdiff below returns the negative of the negative inputs
#         #lob is sigmoid, so odd function, negatives cancel, so not needed
#         if self.lob:
#             return self.lobf(squeezed*ueg_factor)
#         else:
#             return squeezed*ueg_factor
# class LDA_X(torch.nn.Module):
#     def __init__(self):
#         """ UEG exchange"""
#         super().__init__()

#     def forward(self, rho, **kwargs):
#         return -3/4*(3/np.pi)**(1/3)*rho**(1/3)
# params_a_pp     = [1,  1,  1]
# params_a_alpha1 = [0.21370,  0.20548,  0.11125]
# params_a_a      = [0.031091, 0.015545, 0.016887]
# params_a_beta1  = [7.5957, 14.1189, 10.357]
# params_a_beta2  = [3.5876, 6.1977, 3.6231]
# params_a_beta3  = [1.6382, 3.3662,  0.88026]
# params_a_beta4  = [0.49294, 0.62517, 0.49671]
# params_a_fz20   = 1.709921
       
# class PW_C(torch.nn.Module):
#     def __init__(self):
#         """ UEG correlation, Perdew & Wang"""
#         super().__init__()
#     def forward(self, rs, zeta):
#         def g_aux(k, rs):
#             return params_a_beta1[k]*torch.sqrt(rs) + params_a_beta2[k]*rs\
#           + params_a_beta3[k]*rs**1.5 + params_a_beta4[k]*rs**(params_a_pp[k] + 1)

#         def g(k, rs):
#             return -2*params_a_a[k]*(1 + params_a_alpha1[k]*rs)\
#           * torch.log(1 +  1/(2*params_a_a[k]*g_aux(k, rs)))

#         def f_zeta(zeta):
#             return ((1+zeta)**(4/3) + (1-zeta)**(4/3) - 2)/(2**(4/3)-2)

#         def f_pw(rs, zeta):
#             return g(0, rs) + zeta**4*f_zeta(zeta)*(g(1, rs) - g(0, rs) + g(2, rs)/params_a_fz20)\
#           - f_zeta(zeta)*g(2, rs)/params_a_fz20

#         return f_pw(rs, zeta)

# class XC(torch.nn.Module):

#     def __init__(self, grid_models=None, heg_mult=True, pw_mult=True,
#                     level = 1, exx_a=None, epsilon=1e-8):
#         """Defines the XC functional on a grid

#         Args:
#             grid_models (list, optional): list of X_L (local exchange) or C_L (local correlation). Defines the xc-models/enhancement factors. Defaults to None.
#             heg_mult (bool, optional): Use homoegeneous electron gas exchange (multiplicative if grid_models is not empty). Defaults to True.
#             pw_mult (bool, optional): Use homoegeneous electron gas correlation (Perdew & Wang). Defaults to True.
#             level (int, optional): Controls the number of density "descriptors" generated. 1: LDA, 2: GGA, 3:meta-GGA, 4: meta-GGA + electrostatic (nonlocal). Defaults to 1.
#             exx_a (_type_, optional): Exact exchange mixing parameter. Defaults to None.
#             epsilon (float, optional): Offset to avoid div/0 in calculations. Defaults to 1e-8.
#         """

#         super().__init__()
#         self.heg_mult = heg_mult
#         self.pw_mult = pw_mult
#         self.grid_coords = None
#         self.training = True
#         self.level = level
#         self.epsilon = epsilon
#         if level > 3:
#             print('WARNING: Non-local models highly experimental and likely will not work ')
#         self.loge = 1e-5
#         self.s_gam = 1

#         if heg_mult:
#             self.heg_model = LDA_X()
#         if pw_mult:
#             self.pw_model = PW_C()
#         self.grid_models = list(grid_models)
#         if self.grid_models:
#             self.grid_models = torch.nn.ModuleList(self.grid_models)
#         self.model_mult = [1 for m in self.grid_models]

#         if exx_a is not None:
#             self.exx_a = torch.nn.Parameter(torch.Tensor([exx_a]))
#             self.exx_a.requires_grad = True
#         else:
#             self.exx_a = 0

#     def evaluate(self):
#         """Switches self.training flag to False
#         """
#         self.training=False
#     def train(self):
#         """Switches self.training flag to True
#         """
#         self.training=True

#     def add_model_mult(self, model_mult):
#         """_summary_

#         .. todo:: 
#             Unclear what the purpose of this is

#         Args:
#             model_mult (_type_): _description_
#         """
#         del(self.model_mult)
#         self.register_buffer('model_mult',torch.Tensor(model_mult))

#     def add_exx_a(self, exx_a):
#         """Adds exact-exchange mixing parameter after initialization

#         Args:
#             exx_a (float): Exchange mixing parameter
#         """
#         self.exx_a = torch.nn.Parameter(torch.Tensor([exx_a]))
#         self.exx_a.requires_grad = True

#     # Density (rho)
#     def l_1(self, rho):
#         """Level 1 Descriptor -- Creates dimensionless quantity from rho.
#         Eq. 3 in `base paper <https://link.aps.org/doi/10.1103/PhysRevB.104.L161109>`_

#         .. math:: x_0 = \\rho^{1/3}

#         Args:
#             rho (torch.Tensor): density

#         Returns:
#             torch.Tensor: dimensionless density
#         """
#         return rho**(1/3)

#     # Reduced density gradient s
#     def l_2(self, rho, gamma):
#         """Level 2 Descriptor -- Reduced gradient density
#         Eq. 5 in `base paper <https://link.aps.org/doi/10.1103/PhysRevB.104.L161109>`_

#         .. math:: x_2=s=\\frac{1}{2(3\\pi^2)^{1/3}} \\frac{|\\nabla \\rho|}{\\rho^{4/3}}

#         Args:
#             rho (torch.Tensor): density
#             gamma (torch.Tensor): squared density gradient

#         Returns:
#             torch.Tensor: reduced density gradient s
#         """
#         return torch.sqrt(gamma)/(2*(3*np.pi**2)**(1/3)*rho**(4/3)+self.epsilon)

#     # Reduced kinetic energy density alpha
#     def l_3(self, rho, gamma, tau):
#         """Level 3 Descriptor -- Reduced kinetic energy density
#         Eq. 6 in `base paper <https://link.aps.org/doi/10.1103/PhysRevB.104.L161109>`_

#         .. math:: x_3 = \\alpha = \\frac{\\tau-\\tau^W}{\\tau^{unif}},

#         where

#         .. math:: \\tau^W = \\frac{|\\nabla \\rho|^2}{8\\rho}, \\tau^{unif} = \\frac{3}{10} (3\\pi^2)^{2/3}\\rho^{5/3}.

#         Args:
#             rho (torch.Tensor): density
#             gamma (torch.Tensor): squared density gradient
#             tau (torch.Tensor): kinetic energy density

#         Returns:
#             torch.Tensor: reduced kinetic energy density
#         """
#         uniform_factor = (3/10)*(3*np.pi**2)**(2/3)
#         tw = gamma/(8*(rho+self.epsilon))
#         #commented is dpyscflite version, uncommented is xcdiff version
#         #return torch.nn.functional.relu((tau - tw)/(uniform_factor*rho**(5/3)+tw*1e-3 + 1e-12))
#         return (tau - gamma/(8*(rho+self.epsilon)))/(uniform_factor*rho**(5/3)+self.epsilon)

#     # Unit-less electrostatic potential
#     def l_4(self, rho, nl):
#         """Level 4 Descriptor -- Unitless electrostatic potential

#         .. todo:: Figure out what exactly this part is

#         Args:
#             rho (torch.Tensor): density
#             nl (torch.Tensor): some non-local descriptor

#         Returns:
#             torch.nn.functional.relu: _description_
#         """
#         u = nl[:,:1]/((rho.unsqueeze(-1)**(1/3))*self.nl_ueg[:,:1] + self.epsilon)
#         wu = nl[:,1:]/((rho.unsqueeze(-1))*self.nl_ueg[:,1:] + self.epsilon)
#         return torch.nn.functional.relu(torch.cat([u,wu],dim=-1))

#     def get_descriptors(self, rho0_a, rho0_b, gamma_a, gamma_b, gamma_ab,nl_a,nl_b, tau_a, tau_b, spin_scaling = False):
#         """Creates 'ML-compatible' descriptors from the electron density and its gradients, a & b correspond to spin channels

#         Args:
#             rho0_a (torch.Tensor): :math:`\\rho` in spin-channel a
#             rho0_b (torch.Tensor): :math:`\\rho` in spin-channel b
#             gamma_a (torch.Tensor): :math:`|\\nabla \\rho|^2` in spin-channel a 
#             gamma_b (torch.Tensor): :math:`|\\nabla \\rho|^2` in spin-channel b
#             gamma_ab (torch.Tensor): _description_
#             nl_a (torch.Tensor): _description_
#             nl_b (torch.Tensor): _description_
#             tau_a (torch.Tensor): KE density in spin-channel a
#             tau_b (torch.Tensor): KE density in spin-channel b
#             spin_scaling (bool, optional): Flag for spin-scaling. Defaults to False.

#         Returns:
#             _type_: _description_
#         """

#         if not spin_scaling:
#             #If no spin-scaling, calculate polarization and use for X1
#             zeta = (rho0_a - rho0_b)/(rho0_a + rho0_b + self.epsilon)
#             spinscale = 0.5*((1+zeta)**(4/3) + (1-zeta)**(4/3)) # zeta

#         if self.level > 0:  #  LDA
#             if spin_scaling:
#                 descr1 = torch.log(self.l_1(2*rho0_a) + self.loge)
#                 descr2 = torch.log(self.l_1(2*rho0_b) + self.loge)
#             else:
#                 descr1 = torch.log(self.l_1(rho0_a + rho0_b) + self.loge)# rho
#                 descr2 = torch.log(spinscale) # zeta
#             descr = torch.cat([descr1.unsqueeze(-1), descr2.unsqueeze(-1)],dim=-1)
#         if self.level > 1: # GGA
#             if spin_scaling:
#                 descr3a = self.l_2(2*rho0_a, 4*gamma_a) # s
#                 descr3b = self.l_2(2*rho0_b, 4*gamma_b) # s
#                 descr3 = torch.cat([descr3a.unsqueeze(-1), descr3b.unsqueeze(-1)],dim=-1)
#                 descr3 = (1-torch.exp(-descr3**2/self.s_gam))*torch.log(descr3 + 1)
#             else:
#                 descr3 = self.l_2(rho0_a + rho0_b, gamma_a + gamma_b + 2*gamma_ab) # s
#                 #line below in xcdiff, not dpyscfl
#                 descr3 = descr3/((1+zeta)**(2/3) + (1-zeta)**2/3)
#                 descr3 = descr3.unsqueeze(-1)
#                 descr3 = (1-torch.exp(-descr3**2/self.s_gam))*torch.log(descr3 + 1)
#             descr = torch.cat([descr, descr3],dim=-1)
#         if self.level > 2: # meta-GGA
#             if spin_scaling:
#                 descr4a = self.l_3(2*rho0_a, 4*gamma_a, 2*tau_a)
#                 descr4b = self.l_3(2*rho0_b, 4*gamma_b, 2*tau_b)
#                 descr4 = torch.cat([descr4a.unsqueeze(-1), descr4b.unsqueeze(-1)],dim=-1)
#                 #below in xcdiff, not dpyscfl
#                 descr4 = descr4**3/(descr4**2+self.epsilon)
#             else:
#                 descr4 = self.l_3(rho0_a + rho0_b, gamma_a + gamma_b + 2*gamma_ab, tau_a + tau_b)
#                 #next 2 in xcdiff, not dpyscfl
#                 descr4 = 2*descr4/((1+zeta)**(5/3) + (1-zeta)**(5/3))
#                 descr4 = descr4**3/(descr4**2+self.epsilon)

#                 descr4 = descr4.unsqueeze(-1)
#             descr4 = torch.log((descr4 + 1)/2)
#             descr = torch.cat([descr, descr4],dim=-1)
#         if self.level > 3: # meta-GGA + V_estat
#             if spin_scaling:
#                 descr5a = self.l_4(2*rho0_a, 2*nl_a)
#                 descr5b = self.l_4(2*rho0_b, 2*nl_b)
#                 descr5 = torch.log(torch.stack([descr5a, descr5b],dim=-1) + self.loge)
#                 descr5 = descr5.view(descr5.size()[0],-1)
#             else:
#                 descr5= torch.log(self.l_4(rho0_a + rho0_b, nl_a + nl_b) + self.loge)

#             descr = torch.cat([descr, descr5],dim=-1)
#         if spin_scaling:
#             print('spin_scaling')
#             print('descr size -- ', descr.size())
#             descr = descr.view(descr.size()[0],-1,2).permute(2,0,1)
#             print('reshaped descr size --', descr.size())
#         return descr


#     def forward(self, dm):
#         """_summary_

#         Args:
#             dm (torch.Tensor): density matrix

#         Returns:
#             _type_: _description_
#         """
#         Exc = 0
#         if self.grid_models or self.heg_mult:
#             if self.ao_eval.dim()==2:
#                 ao_eval = self.ao_eval.unsqueeze(0)
#             else:
#                 ao_eval = self.ao_eval

#             # Create density (and gradients) from atomic orbitals evaluated on grid
#             # and density matrix
#             # rho[ijsp]: del_i phi del_j phi dm (s: spin, p: grid point index)
#             #print("FORWARD PASS IN XC. AO_EVAL SHAPE, DM SHAPE: ", ao_eval.shape, dm.shape)
#             rho = contract('xij,yik,...jk->xy...i', ao_eval, ao_eval, dm)+1e-10
#             rho0 = rho[0,0]
#             drho = rho[0,1:4] + rho[1:4,0]
#             tau = 0.5*(rho[1,1] + rho[2,2] + rho[3,3])

#             # Non-local electrostatic potential
#             if self.level > 3:
#                 non_loc = contract('mnQ, QP, Pki, ...mn-> ...ki', self.df_3c, self.df_2c_inv, self.vh_on_grid, dm)
#             else:
#                 non_loc = torch.zeros_like(tau).unsqueeze(-1)

#             if dm.dim() == 3: # If unrestricted (open-shell) calculation

#                 # Density
#                 rho0_a = rho0[0]
#                 rho0_b = rho0[1]

#                 # Contracted density gradient
#                 gamma_a, gamma_b = contract('ij,ij->j',drho[:,0],drho[:,0]), contract('ij,ij->j',drho[:,1],drho[:,1])
#                 gamma_ab = contract('ij,ij->j',drho[:,0],drho[:,1])

#                 # Kinetic energy density
#                 tau_a, tau_b = tau

#                 # E.-static
#                 non_loc_a, non_loc_b = non_loc
#             else:
#                 rho0_a = rho0_b = rho0*0.5
#                 gamma_a=gamma_b=gamma_ab= contract('ij,ij->j',drho[:],drho[:])*0.25
#                 tau_a = tau_b = tau*0.5
#                 non_loc_a=non_loc_b = non_loc*0.5

#             # xc-energy per unit particle
#             exc = self.eval_grid_models(torch.cat([rho0_a.unsqueeze(-1),
#                                                     rho0_b.unsqueeze(-1),
#                                                     gamma_a.unsqueeze(-1),
#                                                     gamma_ab.unsqueeze(-1),
#                                                     gamma_b.unsqueeze(-1),
#                                                     torch.zeros_like(rho0_a).unsqueeze(-1), #Dummy for laplacian
#                                                     torch.zeros_like(rho0_a).unsqueeze(-1), #Dummy for laplacian
#                                                     tau_a.unsqueeze(-1),
#                                                     tau_b.unsqueeze(-1),
#                                                     non_loc_a,
#                                                     non_loc_b],dim=-1))
#             print('xc call, exc.shape', exc.shape)
#             #inplace modification throws MulBackwards0 error sometimes?
#             Exc += torch.sum(((rho0_a + rho0_b)*exc.clone()[:,0])*self.grid_weights)
#             #Exc = torch.sum(((rho0_a + rho0_b)*exc[:,0])*self.grid_weights)
#             # try:
#             #     Exc = torch.sum(((rho0_a + rho0_b)*exc[:,0])*self.grid_weights)
#             # except:
#             #     e = sys.exc_info()[0]
#             #     Exc = torch.sum(((rho0_a + rho0_b)*exc[:,0])*self.grid_weights)
#             #     print("Error detected")
#             #     print(e)                

#         #Below in xcdiff, not in dpyscfl
#         #However, keep commented out -- self.nxc_models not implemented
#         #if self.nxc_models:
#         #    for nxc_model in self.nxc_models:
#         #        Exc += nxc_model(dm, self.ml_ovlp)

#         # print('XC.FORWARD: Exc = ', Exc)
        
#         return Exc

#     def eval_grid_models(self, rho, debug=False):
#         """Evaluates all models stored in self.grid_models along with HEG exchange and correlation


#         Args:
#             rho ([list of torch.Tensors]): List with [rho0_a,rho0_b,gamma_a,gamma_ab,gamma_b, dummy for laplacian, dummy for laplacian, tau_a, tau_b, non_loc_a, non_loc_b]

#         Returns:
#             _type_: _description_
#         """
#         Exc = 0
#         rho0_a = rho[:, 0]
#         rho0_b = rho[:, 1]
#         gamma_a = rho[:, 2]
#         gamma_ab = rho[:, 3]
#         gamma_b = rho[:, 4]
#         tau_a = rho[:, 7]
#         tau_b = rho[:, 8]
#         nl = rho[:,9:]
#         nl_size = nl.size()[-1]//2
#         nl_a = nl[:,:nl_size]
#         nl_b = nl[:,nl_size:]

#         C_F= 3/10*(3*np.pi**2)**(2/3)
#         #in xcdiff, self.meta_local would change below assignments
#         #not used here
#         rho0_a_ueg = rho0_a
#         rho0_b_ueg = rho0_b

#         zeta = (rho0_a_ueg - rho0_b_ueg)/(rho0_a_ueg + rho0_b_ueg + 1e-8)
#         rs = (4*np.pi/3*(rho0_a_ueg+rho0_b_ueg + 1e-8))**(-1/3)
#         rs_a = (4*np.pi/3*(rho0_a_ueg + 1e-8))**(-1/3)
#         rs_b = (4*np.pi/3*(rho0_b_ueg + 1e-8))**(-1/3)


#         exc_a = torch.zeros_like(rho0_a)
#         exc_b = torch.zeros_like(rho0_a)
#         exc_ab = torch.zeros_like(rho0_a)

#         if debug:
#             print('eval_grid_models nan summary:')
#             print('zeta, rs, rs_a, rs_b, exc_a, exc_b, exc_ab')
#             print('{}, {}, {}, {}, {}, {}, {}'.format(
#                 torch.isnan(zeta).any().sum(),
#                 torch.isnan(rs).any().sum(),
#                 torch.isnan(rs_a).any().sum(),
#                 torch.isnan(rs_b).any().sum(),
#                 torch.isnan(exc_a).any().sum(),
#                 torch.isnan(exc_b).any().sum(),
#                 torch.isnan(exc_ab).any().sum(),                
#             ))

#         descr_method = self.get_descriptors


#         descr_dict = {}
#         rho_tot = rho0_a + rho0_b
#         if self.grid_models:

#             for grid_model in self.grid_models:
#                 if not grid_model.spin_scaling:
#                     if not 'c' in descr_dict:
#                         descr_dict['c'] = descr_method(rho0_a, rho0_b, gamma_a, gamma_b,
#                                                                          gamma_ab, nl_a, nl_b, tau_a, tau_b, spin_scaling = False)
#                         descr_dict['c'] = descr_method(rho0_a, rho0_b, gamma_a, gamma_b,
#                                                                          gamma_ab, nl_a, nl_b, tau_a, tau_b, spin_scaling = False)
#                     descr = descr_dict['c']
#                     #print("DESCR: ", descr)
#                     #print("DESCR MAX:", torch.max(descr))
#                     #print("DESCR MIN: ", torch.min(descr))
#                     #print("GRID MODEL: ", grid_model)
#                     for name, param in grid_model.named_parameters():
#                         if torch.isnan(param).any():
#                             print("NANS IN NETWORK WEIGHT -- {}".format(name))
#                             raise ValueError("NaNs in Network Weights.")

#                     #Evaluate network with descriptors on grid
#                     #in xcdiff, edge_index is passed here, not in dpyscfl
#                     exc = grid_model(descr,
#                                       grid_coords = self.grid_coords)
#                     #print("EXC GRID_MODEL C: ", exc)

#                     #Included from xcdiff, 2dim exc -> spin polarized
#                     if exc.dim() == 2: #If using spin decomposition
#                         pw_alpha = self.pw_model(rs_a, torch.ones_like(rs_a))
#                         pw_beta = self.pw_model(rs_b, torch.ones_like(rs_b))
#                         pw = self.pw_model(rs, zeta)
#                         ec_alpha = (1 + exc[:,0])*pw_alpha*rho0_a/(rho_tot+1e-8)
#                         ec_beta =  (1 + exc[:,1])*pw_beta*rho0_b/(rho_tot+1e-8)
#                         ec_mixed = (1 + exc[:,2])*(pw*rho_tot - pw_alpha*rho0_a - pw_beta*rho0_b)/(rho_tot+1e-8)
#                         exc_ab = ec_alpha + ec_beta + ec_mixed
#                     else:
#                         if self.pw_mult:
#                             exc_ab += (1 + exc)*self.pw_model(rs, zeta)
#                         else:
#                             exc_ab += exc
# #                    if self.pw_mult:
# #                        exc_ab += (1 + exc)*self.pw_model(rs, zeta)
# #                    else:
# #                        exc_ab += exc
#                 else:
#                     if not 'x' in descr_dict:
#                         descr_dict['x'] = descr_method(rho0_a, rho0_b, gamma_a, gamma_b,
#                                                                          gamma_ab, nl_a, nl_b, tau_a, tau_b, spin_scaling = True)
#                     descr = descr_dict['x']

#                     #in xcdiff, edge_index is passed here, not in dpyscfl
#                     exc = grid_model(descr,
#                                   grid_coords = self.grid_coords)

#                     #print("EXC GRID_MODEL X: ", exc)

#                     if self.heg_mult:
#                         exc_a += (1 + exc[0])*self.heg_model(2*rho0_a_ueg)*(1-self.exx_a)
#                     else:
#                         exc_a += exc[0]*(1-self.exx_a)

#                     if torch.all(rho0_b == torch.zeros_like(rho0_b)): #Otherwise produces NaN's
#                         exc_b += exc[0]*0
#                     else:
#                         if self.heg_mult:
#                             exc_b += (1 + exc[1])*self.heg_model(2*rho0_b_ueg)*(1-self.exx_a)
#                         else:
#                             exc_b += exc[1]*(1-self.exx_a)

#         else:
#             if self.heg_mult:
#                 exc_a = self.heg_model(2*rho0_a_ueg)
#                 exc_b = self.heg_model(2*rho0_b_ueg)
#             if self.pw_mult:
#                 exc_ab = self.pw_model(rs, zeta)


#         # exc = rho0_a_ueg/rho_tot*exc_a + rho0_b_ueg/rho_tot*exc_b + exc_ab
#         exc = exc_a * (rho0_a_ueg/ (rho_tot + self.epsilon)) + exc_b*(rho0_b_ueg / (rho_tot + self.epsilon)) + exc_ab
#         if debug:
#             print('eval_grid_models nan summary:')
#             print('zeta, rs, rs_a, rs_b, exc_a, exc_b, exc_ab')
#             print('{}, {}, {}, {}, {}, {}, {}'.format(
#                 torch.isnan(zeta).any().sum(),
#                 torch.isnan(rs).any().sum(),
#                 torch.isnan(rs_a).any().sum(),
#                 torch.isnan(rs_b).any().sum(),
#                 torch.isnan(exc_a).any().sum(),
#                 torch.isnan(exc_b).any().sum(),
#                 torch.isnan(exc_ab).any().sum(),                
#             ))

#         return exc.unsqueeze(-1)
# class make_rdm1(torch.nn.Module):

#     def __init__(self):
#         """ Generate one-particle reduced density matrix"""
#         super().__init__()

#     def forward(self, mo_coeff, mo_occ):
#         """Forward pass calculating one-particle reduced density matrix.

#         Args:
#             mo_coeff (torch.Tensor/np.array(?)): Molecular orbital coefficients
#             mo_occ (torch.Tensor/np.array(?)): Molecular orbital occupation numbers

#         Returns:
#             torch.Tensor/np.array(?): The RDM1
#         """
#         if mo_coeff.ndim == 3:
#             mocc_a = mo_coeff[0, :, mo_occ[0]>0]
#             mocc_b = mo_coeff[1, :, mo_occ[1]>0]
#             if torch.sum(mo_occ[1]) > 0:
#                 return torch.stack([contract('ij,jk->ik', mocc_a*mo_occ[0,mo_occ[0]>0], mocc_a.T),
#                                     contract('ij,jk->ik', mocc_b*mo_occ[1,mo_occ[1]>0], mocc_b.T)],dim=0)
#             else:
#                 return torch.stack([contract('ij,jk->ik', mocc_a*mo_occ[0,mo_occ[0]>0], mocc_a.T),
#                                     torch.zeros_like(mo_coeff)[0]],dim=0)
#         else:
#             mocc = mo_coeff[:, mo_occ>0]
#             return contract('ij,jk->ik', mocc*mo_occ[mo_occ>0], mocc.T)

# class get_rho(torch.nn.Module):
#     def __init__(self):
#         super().__init__()


#     def forward(self, dm, results):
#         ao_eval = results['ao_eval'][0]
#         print("AO_EVAL, DM SHAPES: {}. {}.".format(ao_eval.shape, dm.shape))
#         if dm.ndim == 2:
#             print("2D DM.")
#             print("RESULTS N_ELEC: ", results['n_elec'])
#             rho = contract('ij,ik,jk->i',
#                                ao_eval, ao_eval, dm)
#         else:
#             print("NON-2D DM")
#             rho = contract('ij,ik,xjk->xi',
#                                ao_eval, ao_eval, dm)
#         return rho

# class energy_tot(torch.nn.Module):

#     def __init__(self):
#         """
#         Total energy (electron-electron + electron-ion; ion-ion not included)
#         """
#         super().__init__()

#     def forward(self, dm, hcore, veff):
#         """Tensor contraction to find total electron energy (e-e + e-ion)

#         Args:
#             dm (torch.Tensor): Density matrix
#             hcore (torch.Tensor): Core Hamiltonian
#             veff (torch.Tensor): Effective Potential

#         Returns:
#             torch.Tensor: The electronic energy
#         """
#         return torch.sum((contract('...ij,ij', dm, hcore) + .5*contract('...ij,...ij', dm, veff))).unsqueeze(0)

# class get_veff(torch.nn.Module):
#     def __init__(self, exx=False, model=None, req_grad=False):
#         """Builds the one-electron effective potential (not including local xc-potential)

#         Args:
#             exx (bool, optional): Exact exchange flag. Defaults to False.
#             model (xc-model): Only used for exact exchange mixing parameter. Defaults to None.
#             df (bool, optional): Use density fitting flag. Defaults to False.
#         """
#         super().__init__()
#         self.exx = exx
#         self.model = model
#         self.req_grad = req_grad
        
#     def forward(self, dm, eri):
#         """Forward pass if no density fitting

#         Args:
#             dm (torch.Tensor): Density matrix
#             eri (torch.Tensor(?)): Electron repulsion integral tensor

#         Returns:
#             torch.Tensor: The "effective" potential
#         """
#         J = contract('...ij,ijkl->...kl',dm, eri)
#         if self.exx:
#             K = self.model.exx_a * contract('...ij,ikjl->...kl',dm, eri)
#         else:
#             K =  torch.zeros_like(J)

#         if J.ndim == 3:
#             return J[0] + J[1] - K
#         else:
#             return J-0.5*K
#     def forward2(self, dm, eri):
#         ''' reimplementation of hf.dot_eri_dm '''
#         nao = dm.shape[-1]
#         if eri.nelement() == nao**4:
#             vj = contract('...ij,ijkl->...kl',dm, eri)
#             if self.exx:
#                 vk = self.model.exx_a * contract('...ij,ikjl->...kl',dm, eri)
#             else:
#                 vk =  torch.zeros_like(vj)
    
#         else:
#             # raise ValueError('eri elements != nao**4')
#             vj, vk = scf._vhf.incore(eri.detach().numpy(), dm.detach().numpy(), 0, with_j = True, with_k = self.exx)

#         if not self.exx:
#             vk = np.zeros_like(vj)
#         if vj.ndim == 3:
#             veff =  vj[0] + vj[1] - vk
#         else:
#             veff =  vj-0.5*vk

#         return torch.tensor(veff, requires_grad=self.req_grad)
#         # if vj.ndim == 3:
#         #     return vj[0] + vj[1] - vk
#         # else:
#         #     return vj - 0.5*vk    
        
        

# def get_veff_np(dm, eri):
#         """Forward pass if no density fitting

#         Args:
#             dm (torch.Tensor): Density matrix
#             eri (torch.Tensor(?)): Electron repulsion integral tensor

#         Returns:
#             torch.Tensor: The "effective" potential
#         """
#         J = contract('...ij,ijkl->...kl',dm, eri)
#         K =  torch.zeros_like(J)
#         if J.ndim == 3:
#             return J[0] + J[1] - K
#         else:
#             return J-0.5*K
# def energy_tot_np(dm, hcore, veff):
#         """Tensor contraction to find total electron energy (e-e + e-ion)

#         Args:
#             dm (torch.Tensor): Density matrix
#             hcore (torch.Tensor): Core Hamiltonian
#             veff (torch.Tensor): Effective Potential

#         Returns:
#             torch.Tensor: The electronic energy
#         """
#         return torch.sum((contract('...ij,ij', dm, hcore) + .5*contract('...ij,...ij', dm, veff))).unsqueeze(0)
# def make_rdm1_np(mo_coeff, mo_occ):
#         """Forward pass calculating one-particle reduced density matrix.

#         Args:
#             mo_coeff (torch.Tensor/np.array(?)): Molecular orbital coefficients
#             mo_occ (torch.Tensor/np.array(?)): Molecular orbital occupation numbers

#         Returns:
#             torch.Tensor/np.array(?): The RDM1
#         """
#         if mo_coeff.ndim == 3:
#             mocc_a = mo_coeff[0, :, mo_occ[0]>0]
#             mocc_b = mo_coeff[1, :, mo_occ[1]>0]
#             if torch.sum(mo_occ[1]) > 0:
#                 return torch.stack([contract('ij,jk->ik', mocc_a*mo_occ[0,mo_occ[0]>0], mocc_a.T),
#                                     contract('ij,jk->ik', mocc_b*mo_occ[1,mo_occ[1]>0], mocc_b.T)],dim=0)
#             else:
#                 return torch.stack([contract('ij,jk->ik', mocc_a*mo_occ[0,mo_occ[0]>0], mocc_a.T),
#                                     torch.zeros_like(mo_coeff)[0]],dim=0)
#         else:
#             mocc = mo_coeff[:, mo_occ>0]
#             return contract('ij,jk->ik', mocc*mo_occ[mo_occ>0], mocc.T)



# def get_fock(hc, veff):
#     """Get the Fock matrix

#     Args:
#         hc (torch.Tensor): Core Hamiltonian
#         veff (torch.Tensor): Effective Potential

#     Returns:
#         torch.Tensor: hc+veff
#     """
#     return hc + veff
# def get_hcore(v, t):
#     """ "Core" Hamiltionian, includes ion-electron and kinetic contributions

#     .. math:: H_{core} = T + V_{nuc-elec}

#     Args:
#         v (torch.Tensor, np.array): Electron-ion interaction energy
#         t (torch.Tensor, np.array): Kinetic energy

#     Returns:
#         torch.Tensor: v + t
#     """
#     return v + t


# class eig(torch.nn.Module):

#     def __init__(self):
#         """Solves generalized eigenvalue problem using Cholesky decomposition
#         """
#         super().__init__()

#     def forward(self, h, s_chol):
#         """Solver for generalized eigenvalue problem

#         .. todo:: torch.symeig is deprecated for torch.linalg.eigh, replace

#         Args:
#             h (torch.Tensor): Hamiltionian
#             s_chol (torch.Tensor): (Inverse) Cholesky decomp. of overlap matrix S
#                                     s_chol = np.linalg.inv(np.linalg.cholesky(S))

#         Returns:
#             (torch.Tensor, torch.Tensor): Eigenvalues (MO energies), eigenvectors (MO coeffs)
#         """
#         #e, c = torch.symeig(contract('ij,...jk,kl->...il',s_chol, h, s_chol.T), eigenvectors=True,upper=False)
#         upper=False
#         UPLO = "U" if upper else "L"
#         e, c = torch.linalg.eigh(contract('ij,...jk,kl->...il',s_chol, h, s_chol.T), UPLO=UPLO)
#         c = contract('ij,...jk ->...ik',s_chol.T, c.clone())
#         return e, c
# torch._C._debug_only_display_vmap_fallback_warnings(True)
# class SCF(torch.nn.Module):

#     def __init__(self, alpha=0.8, nsteps=10, xc=None, device='cpu', exx=False):
#         """This class implements the self-consistent field (SCF) equations

#         Args:
#             alpha (float, optional): Linear mixing parameter. Defaults to 0.8.
#             nsteps (int, optional): Number of scf steps. Defaults to 10.
#             xc (dpyscfl.net.XC, optional): Class containing the exchange-correlation models. Defaults to None.
#             device (str, optional): {'cpu','cuda'}, which device to use. Defaults to 'cpu'.
#             exx (bool, optional): Use exact exchange flag. Defaults to False.
#         """
#         super().__init__()
#         self.nsteps = nsteps
#         self.alpha = alpha
#         self.get_veff = get_veff(exx, xc, req_grad=REQ_GRAD).to(device) # Include Fock (exact) exchange?

#         self.eig = eig().to(device)
#         self.energy_tot = energy_tot().to(device)
#         self.make_rdm1 = make_rdm1().to(device)
#         self.xc = xc
#         #ncore parameter used in xcdiff, not here

#     def forward(self, dm, matrices, sc=True, **kwargs):
#         """Forward pass SCF cycle

#         Args:
#             dm (torch.Tensor): Initial density matrix
#             matrices (dict of torch.Tensors): Contains all other matrices that are considered fixed during SCF calculations (e-integrals etc.)
#             sc (bool, optional): If True does self-consistent calculations, else single-pass. Defaults to True.

#         Returns:
#             dict of torch.Tensors: results: E, dm, and mo_energies
#         """
#         dm = dm[0]

#         # Required matrices
#         # ===================
#         # v: Electron-ion pot.
#         # t: Kinetic
#         # mo_occ: MO occupations
#         # e_nuc: Ion-Ion energy contribution
#         # s: overlap matrix
#         # s_chol: inverse Cholesky decomposition of overlap matrix
#         v, t, mo_occ, e_nuc, s, s_chol = [matrices[key][0] for key in \
#                                              ['v','t','mo_occ',
#                                              'e_nuc','s','s_chol']]
#         hc = get_hcore(v,t)

#         # Optional matrices
#         # ====================

#         # Electron repulsion integrals
#         eri = matrices.get('eri',[None])[0]

#         grid_weights = matrices.get('grid_weights',[None])[0]
#         grid_coords = matrices.get('grid_coords',[None])[0]
#         #edge index called for here in xcdiff, not here

#         # Atomic orbitals evaluated on grid
#         ao_eval = matrices.get('ao_eval',[None])[0]

#         # Used to restore correct potential after symmetrization:
#         L = matrices.get('L', [torch.eye(dm.size()[-1])])[0]
#         scaling = matrices.get('scaling',[torch.ones([dm.size()[-1]]*2)])[0]

#         # Density fitting integrals
#         df_2c_inv = matrices.get('df_2c_inv',[None])[0]
#         df_3c = matrices.get('df_3c',[None])[0]

#         # Electrostatic potential on grid
#         vh_on_grid = matrices.get('vh_on_grid',[None])[0]

#         dm_old = dm

#         E = []
#         deltadm = []
#         nsteps = self.nsteps

#         # if not self.xc.training:
#         #     #if not training, backpropagation doesn't happen so don't need derivatives beyond
#         #     #calculation at a given step
#         #     create_graph = False
#         # else:
#         #     create_graph = True
#         vvv = kwargs.get('verbose', False)
#         if vvv:
#             print('SCF Loop Beginning: {} Steps'.format(nsteps))

#         # SCF iteration loop
#         for step in range(nsteps):
#             #some diis happens here in xcdiff, not implemented here
#             if vvv:
#                 print('Step {}'.format(step))
#             alpha = (self.alpha)**(step)+0.3
#             beta = (1-alpha)
#             dm = alpha * dm + beta * dm_old

#             dm_old = dm
#             if vvv:
#                 print("Density Matrix stats: ")
#                 print("Mean: ", torch.mean(dm))
#                 print("Min/Max: ", torch.min(dm), torch.max(dm))
#                 print("Select Indices: dm.flatten()[[0, 5, 10, 100]]", dm.flatten()[[0,5,10,100]])

#             if df_3c is not None:
#                 veff = self.get_veff.forward_df(dm, df_3c, df_2c_inv, eri)
#             elif kwargs.get('erisym_veff', False):
#                 veff = self.get_veff.forward2(dm, eri)
#             else:
#                 veff = self.get_veff(dm, eri)

#             if kwargs.get('debug', False):
#                 print('STEP-{}/VEFF: '.format(step), veff)
            
#             if self.xc: #If using xc-functional (not Hartree-Fock)
#                 self.xc.ao_eval = ao_eval
#                 self.xc.grid_weights = grid_weights
#                 self.xc.grid_coords = grid_coords
#                 #edge index, ml_ovlp called for here in xcdiff
#                 if vh_on_grid is not None:
#                     self.xc.vh_on_grid = vh_on_grid
#                     self.xc.df_2c_inv = df_2c_inv
#                     self.xc.df_3c = df_3c

#                 if torch.sum(mo_occ) == 1:   # Otherwise H produces NaNs
#                     dm[1] = dm.clone()[0]*1e-12
#                     dm_old[1] = dm.clone()[0]*1e-12

#                 exc = self.xc(dm)

#                 if kwargs.get('debug', False):
#                     print('STEP-{}/exc: '.format(step), exc)

                
#                 # vxc = torch.autograd.functional.jacobian(self.xc, dm, create_graph=True)
#                 vxc = torch.autograd.functional.jacobian(self.xc, dm, create_graph=False,
#                                                          vectorize=False)
#                 vxc1 = torch.autograd.grad(exc, dm)[0]
#                 print('vxc/vxc1 shapes,', vxc.shape, vxc1.shape)
#                 if kwargs.get('debug', False):
#                     msize = vxc.element_size() * vxc.nelement()
#                     msize1 = vxc1.element_size() * vxc1.nelement()
#                     print('vxc: SHAPE = {}. SIZE = {} KB / {} MB / {} GB'.format(k, vxc.shape, msize/(1000), msize/(1000**2), msize/(1000**3)))
#                     print('vxc1: SHAPE = {}. SIZE = {} KB / {} MB / {} GB'.format(k, vxc1.shape, msize1/(1000), msize1/(1000**2), msize1/(1000**3)))
#                     print('|vxc - vxc1|.max(): ', abs(vxc-vxc1).max())
#                 # Restore correct symmetry for vxc
#                 if vxc.dim() > 2:
#                     vxc = contract('ij,xjk,kl->xil',L,vxc.clone(),L.T)
#                     vxc = torch.where(scaling.unsqueeze(0) > 0 , vxc.clone(), scaling.unsqueeze(0))
#                 else:
#                     vxc = torch.mm(L,torch.mm(vxc.clone(),L.T))
#                     vxc = torch.where(scaling > 0 , vxc.clone(), scaling)

#                 if torch.sum(mo_occ) == 1:   # Otherwise H produces NaNs
#                     vxc[1] = torch.zeros_like(vxc.clone()[1])

#                 veff += vxc

#                 if kwargs.get('debug', False):
#                     print('STEP-{}/VEFF+VXC: '.format(step), veff)


#                 #Add random noise to potential to avoid degeneracies in EVs
#                 if self.xc.training:#: and sc:
#                     if step == 0:
#                         print("Noise generation to avoid potential degeneracies")
#                     noise = torch.abs(torch.randn(vxc.size(),device=vxc.device)*1e-4)
#                     noise = noise + torch.transpose(noise,-1,-2)
#                     veff = veff.clone() + noise
#                 if kwargs.get('debug', False):
#                     print('STEP-{}/VEFF+VXC+NOISE: '.format(step), veff)

#             else:
#                 exc=0
#                 vxc=torch.zeros_like(veff)
#             f = get_fock(hc, veff)
#             if kwargs.get('debug', False):
#                 print('STEP-{}/FOCK: '.format(step), f)

#             mo_e, mo_coeff = self.eig(f, s_chol)
#             dm = self.make_rdm1(mo_coeff, mo_occ)

#             # e_tot = self.energy_tot(dm_old, hc, veff-vxc)+ e_nuc + exc
#             e_tot = self.energy_tot(dm, hc, veff-vxc)+ e_nuc + exc
#             E.append(e_tot)
#             if vvv:
#                 print("{} Energy: {}".format(step, e_tot))
#                 print("History: {}".format(E))
#             if not sc:
#                 break

#         #in xcdiff, things happen here with mo_occ[:self.ncore], e_ip etc. not implemented here
        
#         results = {'E': torch.cat(E), 'dm':dm, 'mo_energy':mo_e}

#         return results

# def get_optimizer(model, path='', hybrid=None, lr=1e-3, l2=1e-6):
#     if hybrid:
#             optimizer = torch.optim.Adam(list(model.parameters()) + [model.xc.exx_a],
#                                     lr=lr, weight_decay=l2)
#     else:
#         optimizer = torch.optim.Adam(model.parameters(),
#                                     lr=lr, weight_decay=l2)

#     MIN_RATE = 1e-7
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',
#                                                             verbose=True, patience=int(10/PRINT_EVERY),
#                                                             factor=0.1, min_lr=MIN_RATE)

#     if path:
#         optimizer.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
#     return optimizer, scheduler

In [4]:
def get_torch_xc(xctype, pretrain_loc='', hyb_par=0, path='', DEVICE='cpu', ueg_limit=True, meta_x=None, freec=False,
            inserts = 0, nhidden = 16):
    """_summary_

    Args:
        xctype (_type_): _description_
        pretrain_loc (_type_): _description_
        hyb_par (int, optional): _description_. Defaults to 0.
        path (str, optional): _description_. Defaults to ''.
        DEVICE (str, optional): _description_. Defaults to 'cpu'.
        ueg_limit (bool, optional): _description_. Defaults to True.
        meta_x (_type_, optional): _description_. Defaults to None.
        freec (bool, optional): _description_. Defaults to False.
    """
    print('FREEC', freec)
    if xctype == 'GGA':
        lob = 1.804 if ueg_limit else 0
        x = X_L(device=DEVICE,n_input=1, n_hidden=nhidden, use=[1], lob=lob, ueg_limit=ueg_limit) # PBE_X
        c = C_L(device=DEVICE,n_input=3, n_hidden=nhidden, use=[2], ueg_limit=ueg_limit and not freec)
        xc_level = 2
    elif xctype == 'MGGA':
        lob = 1.174 if ueg_limit else 0
        x = X_L(device=DEVICE,n_input=2, n_hidden=nhidden, use=[1,2], lob=1.174, ueg_limit=ueg_limit) # PBE_X
        c = C_L(device=DEVICE,n_input=4, n_hidden=nhidden, use=[2,3], ueg_limit=ueg_limit and not freec)
        xc_level = 3
    if pretrain_loc:
        print("Loading pre-trained models from " + pretrain_loc)
        x.load_state_dict(torch.load(pretrain_loc + '/x'))
        c.load_state_dict(torch.load(pretrain_loc + '/c'))
    EXX = bool(hyb_par)
    EXX_A = hyb_par if hyb_par else None

    xc = XC(grid_models=[x, c], heg_mult=True, level=xc_level)
    if path:
        try:
            xcp = torch.load(path, map_location=torch.device('cpu')).xc
            xc.load_state_dict(xcp.state_dict())
        except AttributeError:
            # AttributeError: 'RecursiveScriptModule' object has no attribute 'copy'
            #occurs when loading finished xc from xcdiff
            xcp = torch.jit.load(path)
            xc.load_state_dict(xcp.state_dict())

    return xc
def get_torch_weights_and_biases(torch_net):
    weights = []
    biases = []
    for nidx, net in enumerate(torch_net):
        try:
            w = jnp.array(net.weight.data)
            b = jnp.array(net.bias.data)
            weights.append(w)
            biases.append(b)
        except:
            print('This torch layer is not a Linear model.')
            continue
    return (weights, biases)
    
#per https://docs.kidger.site/equinox/tricks/
def trunc_init(weight: jax.Array, key: jax.random.PRNGKey) -> jax.Array:
    out, in_ = weight.shape
    stddev = math.sqrt(1 / in_)
    return stddev * jax.random.truncated_normal(key, shape=(out, in_), lower=-2, upper=2)

def init_linear_weight(model, seed, new_weights, new_bias):
    jax.random.PRNGKey(seed)
    is_linear = lambda x: isinstance(x, eqx.nn.Linear)
    get_weights = lambda m: [x.weight
                           for x in jax.tree_util.tree_leaves(m, is_leaf=is_linear)
                           if is_linear(x)]
    get_bias = lambda m: [x.bias
                           for x in jax.tree_util.tree_leaves(m, is_leaf=is_linear)
                           if is_linear(x)]

    weights = get_weights(model)
    bias = get_bias(model)
    new_model = eqx.tree_at(get_weights, model, new_weights)
    new_model = eqx.tree_at(get_bias, new_model, new_bias)
    return new_model

In [5]:
def jax_exc_func(model, ao_eval, gw):
    def ret_func(inp):
        return model(inp, ao_eval, gw)
    return ret_func

def jax_loss_func(loss_func, model, en, ao, gw, eri, mooc, hc, s):
    def ret_func(dm):
        return loss_func(model, dm, en, ao, gw, eri, mooc, hc, s)
    return ret_func

# @eqx.filter_jit
def jax_dm(dm, eri, vxc_grad_func, mo_occ, hc, s, ogd, alpha0=0.7):
    L = jnp.eye(dm.shape[-1])
    scaling = jnp.ones([dm.shape[-1]]*2)
    dm_old = dm
    def true_func(vxc):
        vxc.at[1].set(jnp.zeros_like(vxc[1]))
        return vxc
    def false_func(vxc):
        return vxc
    alpha = jnp.power(alpha0, 0)+0.3
    beta = (1-alpha)
    dm = alpha * dm + beta * dm_old
    dm_old = dm
    veff = xce.utils.get_veff()(dm, eri)
    vxc = jax.grad(vxc_grad_func)(dm)
    if vxc.ndim > 2:
        vxc = jnp.einsum('ij,xjk,kl->xil',L,vxc,L.T)
        vxc = jnp.where(jnp.expand_dims(scaling, 0) > 0 , vxc, jnp.expand_dims(scaling,0))
    else:
        vxc = jnp.matmul(L,jnp.matmul(vxc ,L.T))
        vxc = jnp.where(scaling > 0 , vxc, scaling)
    
    jax.lax.cond(jnp.sum(mo_occ) == 1, true_func, false_func, vxc)
    
    veff += vxc
    f = xce.utils.get_fock()(hc, veff)
    mo_e, mo_c = xce.utils.eig()(f+1e-6*jax.random.uniform(key=jax.random.PRNGKey(92017), shape=f.shape), s, ogd)
    dm = xce.utils.make_rdm1()(mo_c, mo_occ)
    return dm, mo_e, mo_c
    
# @eqx.filter_grad
def e_loss(model, inp_dm, ref_en, ao_eval, grid_weights, *args):
    print(f"e_loss; input stats. inp_dm.shape = {inp_dm.shape}, ref_en = {ref_en}, ao_eval.shape = {ao_eval.shape}, grid_weights.shape = {grid_weights.shape}")
    e_pred = model(inp_dm, ao_eval, grid_weights)
    eL = jnp.sqrt( np.mean((e_pred-ref_en)**2))
    # print('energy loss', eL)
    return eL

class E_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, inp_dm, ref_en, ao_eval, grid_weights):

        e_pred = model(inp_dm, ao_eval, grid_weights)
        eL = jnp.sqrt( jnp.mean((e_pred-ref_en)**2))
        return eL

def holo_loss(model, inp_dm, ref_en, ao_eval, grid_weights, vxc_grad_func, mo_occ, hc, s, eri, ogd, alpha0):
    dm, mo_e, mo_c = jax_dm(inp_dm, eri, vxc_grad_func, mo_occ, hc, s, ogd, alpha0)
    homo_i = jnp.max(jnp.nonzero(mo_occ, size=dm.shape[0])[0])
    homo_e = mo_e[homo_i]
    lumo_e = mo_e[homo_i+1]
    pred_holo = lumo_e - homo_e
    print('pred_holo', pred_holo)
    return jnp.sqrt( np.mean ((pred_holo - ref_en)**2))

def loop_e_loss(model, inp_dms, ref_ens, ao_evals, grid_weights):
    e_preds = []
    for idx in range(len(ref_ens)):
        ep = model(inp_dms[idx], ao_evals[idx], grid_weights[idx])
        e_preds.append(ep)
    e_preds = jnp.array(e_preds)
    e_refs = jnp.array(ref_ens)
    eL = jnp.sqrt( jnp.mean( (e_refs-e_preds)**2))
    return eL
# @eqx.filter_grad

def dm_loss(model, inp_dm, ref_en, ao_eval, gw, eri, mo_occ, hc, s, ogd, *args):
    dmp, moe, moc = jax_dm(inp_dm, eri, jax_exc_func(model, ao_eval, gw), mo_occ, hc, s, ogd)
    dmL = jnp.sqrt(jnp.sum( (dmp - inp_dm)**2))
    return dmL


def loop_dm_loss(model, inp_dms, eris, mo_occs, hcs, ss, ao_evals, gws):
    dmL = 0
    for idx, dm in enumerate(inp_dms):
        dmp = jax_dm(inp_dms[idx], eris[idx], jax_exc_func(model, ao_evals[idx], gws[idx]), mo_occs[idx], hcs[idx], ss[idx])
        dmL += jnp.mean((dmp - inp_dms[idx])**2)
    dmL = jnp.sqrt(dmL)
    return dmL
    
# @eqx.filter_value_and_grad
def total_loss(model, inp_dms, ref_ens, ref_holos, ao_evals, grid_weights, eris, mo_occs, hcs, ss, ogd):
    # eL = e_loss(model, inp_dms, ref_ens, ao_evals, grid_weights, ogd)
    # dmL = dm_loss(model, inp_dms, ref_ens, ao_eval, grid_weights, eris, mo_occs, hcs, ss, ogd)
    vxcgf = jax_exc_func(model, ao_eval, grid_weights)
    holoL = holo_loss(model, inp_dms, ref_holos, ao_evals, grid_weights, vxcgf, mo_occs, hcs, ss, eris, ogd, alpha0=0.7)
    # return jnp.sqrt( eL**2 + holoL**2)
    return jnp.sqrt( holoL**2 )

def total_loop_loss(model, inp_dms, ref_ens, ao_evals, grid_weights, eris, mo_occs, hcs, ss):
    eL = loop_e_loss(model, inp_dms, ref_ens, ao_evals, grid_weights)
    dmL = loop_dm_loss(model, inp_dms, eris, mo_occs, hcs, ss, ao_evals, grid_weights)
    return jnp.sqrt(eL**2 + dmL**2)


Dev for multi-NL input networks

In [7]:
class eX(eqx.Module):
    n_input: int
    n_hidden: int
    ueg_limit: jax.Array
    spin_scaling: bool
    lob: jax.Array
    use: list
    net: eqx.Module
    tanh: jax.named_call
    lobf: jax.named_call
    sig: jax.named_call
    shift: jax.Array
    lobf: eqx.Module
    seed: int
    depth: int

    def __init__(self, n_input, n_hidden=16, depth=3, use=[], ueg_limit=False, lob=1.804, seed=92017):
        """
        __init__ Local exchange model based on MLP.

        Receives density descriptors in this order : [rho, s, alpha, nl], where the input may be truncated depending on XC-level of approximation.

        The MLP generated is hard-coded to have one output value -- the predicted exchange energy given a specific input from the grid.

        :param n_input: Input dimensions (LDA: 1, GGA: 2, meta-GGA: 3, ...)
        :type n_input: int
        :param n_hidden: Number of hidden nodes (three hidden layers used by default), defaults to 16
        :type n_hidden: int, optional
        :param depth: Depth of the MLP, defaults to 3
        :type depth: int, optional
        :param use: Only these indices are used as input to the model (can be used to omit density as input to enforce uniform density scaling). These indices are also used to enforce UEG where the assumed order is [s, alpha, ...], defaults to []
        :type use: list, optional
        :param ueg_limit: Flag to determine whether or not to enforce uniform homoegeneous electron gas limit, defaults to False
        :type ueg_limit: bool, optional
        :param lob: Enforce this value as local Lieb-Oxford bound (don't enforce if set to 0), defaults to 1.804
        :type lob: float, optional
        :param seed: Random seed used to generate initial weights and biases for the MLP, defaults to 92017
        :type seed: int, optional
        """
        super().__init__()
        self.ueg_limit = ueg_limit
        self.spin_scaling = True
        self.lob = lob
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.seed = seed
        self.depth = depth

        if not use:
            self.use = jnp.arange(n_input)
        else:
            self.use = use
        self.net =  eqx.nn.MLP(in_size = self.n_input,
                               out_size = 1,
                               width_size = self.n_hidden,
                               depth = self.depth,
                               activation = jax.nn.gelu,
                              key=jax.random.PRNGKey(self.seed))
        
        self.tanh = jnp.tanh
        self.lobf = xce.net.LOB(limit=self.lob)
        self.sig = jax.nn.sigmoid
        self.shift = 1/(1+jnp.exp(-1e-3))

    def __call__(self, rho, **kwargs):
        """
        __call__ Forward pass for the exchange network.

        Uses :jax.vmap: to vectorize evaluation of the MLP on the descriptors, assuming a shape [batch, *, n_input]

        .. todo: Make sure the :vmap: call can work with specific :use: values beyond the defaults assumed in the previous implementation.

        :param rho: The descriptors to the MLP -- transformed densities and gradients appropriate to the XC-level. This network will only use the dimensions specified in self.use.
        :type rho: jax.Array
        :return: The exchange energy on the grid
        :rtype: jax.Array
        """
        if self.n_input < 4:
            squeezed = jnp.squeeze(jax.vmap(jax.vmap(self.net), in_axes=1)(rho[...,self.use])).T
        else:
            print(f"eX.__call__, rho shape: {rho.shape}")
            squeezed = jnp.squeeze(jax.vmap(self.net)(rho))

        if self.ueg_limit:
            ueg_lim = rho[...,self.use[0]]
            if len(self.use) > 1:
                ueg_lim_a = jnp.power(self.tanh(rho[...,self.use[1]]),2)
            else:
                ueg_lim_a = 0
            if len(self.use) > 2:
                ueg_lim_nl = jnp.sum(rho[...,self.use[2:]],axis=-1)
            else:
                ueg_lim_nl = 0
        else:
            ueg_lim = 1
            ueg_lim_a = 0
            ueg_lim_nl = 0

        if self.lob:
            result = self.lobf(squeezed*(ueg_lim + ueg_lim_a + ueg_lim_nl))
        else:
            result = squeezed*(ueg_lim + ueg_lim_a + ueg_lim_nl)

        return result

class eC(eqx.Module):
    n_input: int
    n_hidden: int
    ueg_limit: jax.Array
    spin_scaling: bool
    lob: jax.Array
    use: list
    net: eqx.Module
    tanh: jax.named_call
    lobf: jax.named_call
    sig: jax.named_call
    lobf: eqx.Module
    seed: int
    depth: int

    def __init__(self, n_input=2,n_hidden=16, depth=3, use = [], ueg_limit=False, lob=2.0, seed=92017):
        """
        __init__ Local correlation model based on MLP.

        Receives density descriptors in this order : [rho, spinscale, s, alpha, nl], where the input may be truncated depending on XC-level of approximation

        .. todo: Make sure the :vmap: call can work with specific :use: values beyond the defaults assumed in the previous implementation.

        :param n_input: Input dimensions (LDA: 2, GGA: 3 , meta-GGA: 4), defaults to 2.
        :type n_input: int
        :param n_hidden: Number of hidden nodes (three hidden layers used by default), defaults to 16
        :type n_hidden: int, optional
        :param depth: Depth of the MLP, defaults to 3
        :type depth: int, optional
        :param use: Only these indices are used as input to the model. These indices are also used to enforce UEG where the assumed order is [s, alpha, ...], defaults to []
        :type use: list, optional
        :param ueg_limit: Flag to determine whether or not to enforce uniform homoegeneous electron gas limit, defaults to False
        :type ueg_limit: bool, optional
        :param lob: Enforce this value as local Lieb-Oxford bound (don't enforce if set to 0), defaults to 2.0
        :type lob: float, optional
        :param seed: Random seed used to generate initial weights and biases for the MLP, defaults to 92017
        :type seed: int, optional
        """
        super().__init__()
        self.spin_scaling = False
        self.lob = False
        self.ueg_limit = ueg_limit
        self.n_input=n_input
        self.n_hidden=n_hidden
        self.seed = seed
        self.depth = depth

        if not use:
            self.use = jnp.arange(n_input)
        else:
            self.use = use
        self.net =  eqx.nn.MLP(in_size = self.n_input,
                               out_size = 1,
                               width_size = self.n_hidden,
                               depth = self.depth,
                               activation = jax.nn.gelu,
                               final_activation = jax.nn.softplus,
                               key=jax.random.PRNGKey(self.seed))
        self.sig = jax.nn.sigmoid
        self.tanh = jnp.tanh
        self.lob = lob
        if self.lob:
            self.lobf = xce.net.LOB(self.lob)
        else:
            self.lob =  1000.0
            self.lobf = xce.net.LOB(self.lob)


    def __call__(self, rho, **kwargs):
        """
        __call__ Forward pass for the correlation network.

        Uses :jax.vmap: to vectorize evaluation of the MLP on the descriptors, assuming a shape [*, n_input]

        :param rho: The descriptors to the MLP -- transformed densities and gradients appropriate to the XC-level. This network will only use the dimensions specified in self.use in determining the UEG limits.
        :type rho: jax.Array
        :return: The exchange energy on the grid
        :rtype: jax.Array
        """
        print(f"eC.__call__, rho shape: {rho.shape}")
        squeezed = jnp.squeeze(-jax.vmap(self.net)(rho))

        if self.ueg_limit:
            ueg_lim = self.tanh(rho[...,self.use[0]])
            if len(self.use) > 1:
                ueg_lim_a = jnp.pow(self.tanh(rho[...,self.use[1]]),2)
            else:
                ueg_lim_a = 0
            if len(self.use) > 2:
                ueg_lim_nl = jnp.sum(self.tanh(rho[...,self.use[2:]])**2,axis=-1)
            else:
                ueg_lim_nl = 0

            ueg_factor = ueg_lim + ueg_lim_a + ueg_lim_nl
        else:
            ueg_factor = 1
        if self.lob:
            return self.lobf(squeezed*ueg_factor)
        else:
            return squeezed*ueg_factor

class eXC(eqx.Module):
    grid_models: list
    heg_mult: bool
    pw_mult: bool
    level: int
    exx_a: jax.Array
    epsilon: jax.Array
    loge: jax.Array
    s_gam: jax.Array
    heg_model: eqx.Module
    pw_model: eqx.Module
    model_mult: list
    debug: bool
    nlstart_i: int
    nlend_i: int
    
    def __init__(self, grid_models=[], heg_mult=True, pw_mult=True,
                    level = 1, exx_a=None, epsilon=1e-8, debug=False,
                nlstart_i = 3, nlend_i = 12):
        """
        __init__ Defines the XC functional

        Constructed with two MLPs -- one for the local exchange energy on the grid, the other for the local correlation energy.

        :param grid_models: list of eX (local exchange) or eC (local correlation). Defines the xc-models/enhancement factors, defaults to []
        :type grid_models: list, optional
        :param heg_mult: Use homoegeneous electron gas exchange (multiplicative if grid_models is not empty), defaults to True
        :type heg_mult: bool, optional
        :param pw_mult: Use homoegeneous electron gas correlation (Perdew & Wang), defaults to True
        :type pw_mult: bool, optional
        :param level: Controls the number of density "descriptors" generated. 1: LDA, 2: GGA, 3:meta-GGA, 4: meta-GGA + electrostatic (nonlocal), defaults to 1
        :type level: int, optional
        :param exx_a: Exact exchange mixing parameter, defaults to None
        :type exx_a: float, optional
        :param epsilon: Offset to avoid div/0 in calculations, defaults to 1e-8
        :type epsilon: float, optional
        :param debug: Controls printing of various stats throughout, defaults to False
        :type debug: bool, optional
        :param nlstart_i: If level > 3, this controls the number of CIDER Nonlocal parameters are selected, defaults to 3 as the first nonlocal CIDER descriptor
        :type nlstart_i: int, optional
        :param nlend_i: If level > 3, this controls the number of CIDER Nonlocal parameters are selected, defaults to 12 as the last nonlocal CIDER descriptor
        :type nlend_i: int, optional

        """
        super().__init__()
        self.heg_mult = heg_mult
        self.pw_mult = pw_mult
        self.level = level
        self.grid_models = grid_models
        self.epsilon = epsilon
        if level > 3:
            print('WARNING: Non-local models highly experimental and likely will not work ')
        self.loge = 1e-5
        self.s_gam = 1
        self.debug = debug
        self.nlstart_i = 3
        self.nlend_i = 12

        if heg_mult:
            self.heg_model = xce.xc.LDA_X()
        if pw_mult:
            self.pw_model = xce.xc.PW_C()
        self.model_mult = [1 for m in self.grid_models]
        if not exx_a:
            self.exx_a = 0
        else:
            self.exx_a = exx_a

    def __call__(self, dm, ao_eval, grid_weights, mf = None):
        """
        __call__ Forward call for the XC network to get the grid point e_xc

        Generates the density-on-grid from the density matrix, atomic orbital evaluation, and the grid weights from a :pyscfad: calculation.


        :param dm: Density matrix
        :type dm: jax.Array
        :param ao_eval: Atomic orbitals evaluated on the grid
        :type ao_eval: jax.Array
        :param grid_weights: Grid weights associated to the grid on which the atomic orbitals are evaluated
        :type grid_weights: jax.Array
        :return: Exc, exchange-correlation energy from integrating the network calls across the grid
        :rtype: float
        """
        Exc = 0
        if self.level > 3:
            assert mf is not None
        if self.grid_models or self.heg_mult:
            if ao_eval.ndim==2:
                ao_eval = jnp.expand_dims(ao_eval,0)
            else:
                ao_eval = ao_eval

            # Create density (and gradients) from atomic orbitals evaluated on grid
            # and density matrix
            # rho[ijsp]: del_i phi del_j phi dm (s: spin, p: grid point index)
            rho = jnp.einsum('xij,yik,...jk->xy...i', ao_eval, ao_eval, dm)+1e-10
            
            rho0 = rho[0,0]
            drho = rho[0,1:4] + rho[1:4,0]
            tau = 0.5*(rho[1,1] + rho[2,2] + rho[3,3])

            non_loc = jnp.zeros_like(tau)

            if dm.ndim == 3: # If unrestricted (open-shell) calculation

                # Density
                rho0_a = rho0[0]
                rho0_b = rho0[1]

                # jnp.einsumed density gradient
                gamma_a, gamma_b = jnp.einsum('ij,ij->j',drho[:,0],drho[:,0]), jnp.einsum('ij,ij->j',drho[:,1],drho[:,1])
                gamma_ab = jnp.einsum('ij,ij->j',drho[:,0],drho[:,1])

                # Kinetic energy density
                tau_a, tau_b = tau

                # E.-static
                non_loc_a, non_loc_b = non_loc
            else:
                rho0_a = rho0_b = rho0*0.5
                gamma_a=gamma_b=gamma_ab= jnp.einsum('ij,ij->j',drho[:],drho[:])*0.25
                tau_a = tau_b = tau*0.5
                non_loc_a=non_loc_b = non_loc*0.5

            # xc-energy per unit particle
            exc = self.eval_grid_models(jnp.concatenate([jnp.expand_dims(rho0_a,-1),
                                                    jnp.expand_dims(rho0_b,-1),
                                                    jnp.expand_dims(gamma_a,-1),
                                                    jnp.expand_dims(gamma_ab,-1),
                                                    jnp.expand_dims(gamma_b,-1),
                                                    jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                    jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                    jnp.expand_dims(tau_a,-1),
                                                    jnp.expand_dims(tau_b,-1),
                                                    jnp.expand_dims(non_loc_a,-1),
                                                    jnp.expand_dims(non_loc_b,-1)],axis=-1),
                                       mf = mf, dm = dm)
            Exc += jnp.sum(((rho0_a + rho0_b)*exc[:,0])*grid_weights)
        return Exc
            
    # Density (rho)
    def l_1(self, rho):
        """
        l_1 Level 1 (LDA-level) Descriptor -- Creates dimensionless quantity from rho.

        Equation 3 from the `base paper`_.

        .. _base paper: https://link.aps.org/doi/10.1103/PhysRevB.104.L161109

        .. math:: x_0 = \\rho^{1/3}


        :param rho: density
        :type rho: jax.Array
        :return: Scaled density
        :rtype: jax.Array
        """
        return rho**(1/3)

    # Reduced density gradient s
    def l_2(self, rho, gamma):
        """
        l_2 Level 2 (GGA-level) Descriptor -- Reduced gradient density

        Equation 5 from the `base paper`_.

        .. _base paper: https://link.aps.org/doi/10.1103/PhysRevB.104.L161109

        .. math:: x_2=s=\\frac{1}{2(3\\pi^2)^{1/3}} \\frac{|\\nabla \\rho|}{\\rho^{4/3}}

        
        :param rho: density
        :type rho: jax.Array
        :param gamma: squared density gradient
        :type gamma: jax.Array
        :return: reduced density gradient s
        :rtype: jax.Array
        """
        return jnp.sqrt(gamma)/(2*(3*np.pi**2)**(1/3)*rho**(4/3)+self.epsilon)

    # Reduced kinetic energy density alpha
    def l_3(self, rho, gamma, tau):
        """
        l_3 Level 3 (MGGA-level) Descriptor -- Reduced kinetic energy density

        Equation 6 from the `base paper`_.

        .. _base paper: https://link.aps.org/doi/10.1103/PhysRevB.104.L161109

        .. math:: \\tau^W = \\frac{|\\nabla \\rho|^2}{8\\rho}, \\tau^{unif} = \\frac{3}{10} (3\\pi^2)^{2/3}\\rho^{5/3}.


        :param rho: density
        :type rho: jax.Array
        :param gamma: squared density gradient
        :type gamma: jax.Array
        :param tau: kinetic energy density
        :type tau: jax.Array
        :return: reduced kinetic energy density
        :rtype: jax.Array
        """
        uniform_factor = (3/10)*(3*np.pi**2)**(2/3)
        tw = gamma/(8*(rho+self.epsilon))
        return (tau - tw)/(uniform_factor*rho**(5/3)+self.epsilon)

    # Unit-less electrostatic potential
    def l_4(self, rho, nl):
        """
        l_4 Level 4 (Non-local level) Descriptor -- Unitless electrostatic potential

        .. todo:: document/implement in a more descriptive manner

        :param rho: density
        :type rho: jax.Array
        :param nl: non-local values arising from density contractions
        :type nl: jax.Array
        :return: the non-local descriptors
        :rtype: jax.Array
        """
        u = nl[:,:1]/((jnp.expand_dims(rho, -1)**(1/3))*self.nl_ueg[:,:1] + self.epsilon)
        wu = nl[:,1:]/((jnp.expand_dims(rho, -1))*self.nl_ueg[:,1:] + self.epsilon)
        return jax.nn.relu(jnp.concatenate([u,wu],axis=-1))

    def nl_4(self, mf, dm):
        an = RKSAnalyzer(mf)
        descr5 = jnp.asarray(get_exchange_descriptors2(an, restricted=True, version='c', auxbasis=mf.mol.basis,
                                   rdm1=True, dm=np.asarray(dm), inmol=True, mol=mf.mol, ingrid=True, grid=mf.grids))
        descr5 = descr5[self.nlstart_i:self.nlend_i]
        return descr5
        
    # @eqx.filter_jit
    def get_descriptors(self, rho0_a, rho0_b, gamma_a, gamma_b, gamma_ab,nl_a,nl_b, tau_a, tau_b, spin_scaling = False,
                       mf = None, dm = None):
        """
        get_descriptors Creates 'ML-compatible' descriptors from the electron density and its gradients, a & b correspond to spin channels

        :param rho0_a: :math:`\\rho` in spin-channel a
        :type rho0_a: jax.Array
        :param rho0_b: :math:`\\rho` in spin-channel b
        :type rho0_b: jax.Array
        :param gamma_a: :math:`|\\nabla \\rho|^2` in spin-channel b
        :type gamma_a: jax.Array
        :param gamma_b: :math:`|\\nabla \\rho|^2` in spin-channel b
        :type gamma_b: jax.Array
        :param gamma_ab: :math:`|\\nabla \\rho|^2`, contracted from both spin channels
        :type gamma_ab: jax.Array
        :param nl_a: Non-local descriptors in spin-channel a, not currently used.
        :type nl_a: jax.Array
        :param nl_b: Non-local descriptors in spin-channel b, not currently used.
        :type nl_b: jax.Array
        :param tau_a: KE density in spin-channel a
        :type tau_a: jax.Array
        :param tau_b: KE density in spin-channel b
        :type tau_b: jax.Array
        :param spin_scaling: Flag for spin-scaling, defaults to False
        :type spin_scaling: bool, optional
        :return: Array of the machine-learning descriptors on the grid
        :rtype: jax.Array
        """
        if not spin_scaling:
            #If no spin-scaling, calculate polarization and use for X1
            zeta = (rho0_a - rho0_b)/(rho0_a + rho0_b + self.epsilon)
            spinscale = 0.5*((1+zeta)**(4/3) + (1-zeta)**(4/3)) # zeta

        if self.level > 0:  #  LDA
            if spin_scaling:
                descr1 = jnp.log(self.l_1(2*rho0_a) + self.loge)
                descr2 = jnp.log(self.l_1(2*rho0_b) + self.loge)
            else:
                descr1 = jnp.log(self.l_1(rho0_a + rho0_b) + self.loge)# rho
                descr2 = jnp.log(spinscale) # zeta
            descr = jnp.concatenate([jnp.expand_dims(descr1, -1), jnp.expand_dims(descr2, -1)],axis=-1)
        if self.level > 1: # GGA
            if spin_scaling:
                descr3a = self.l_2(2*rho0_a, 4*gamma_a) # s
                descr3b = self.l_2(2*rho0_b, 4*gamma_b) # s
                descr3 = jnp.concatenate([jnp.expand_dims(descr3a,-1), jnp.expand_dims(descr3b,-1)],axis=-1)
                descr3 = (1-jnp.exp(-descr3**2/self.s_gam))*jnp.log(descr3 + 1)
            else:
                descr3 = self.l_2(rho0_a + rho0_b, gamma_a + gamma_b + 2*gamma_ab) # s
                descr3 = descr3/((1+zeta)**(2/3) + (1-zeta)**2/3)
                descr3 = jnp.expand_dims(descr3,-1)
                descr3 = (1-jnp.exp(-descr3**2/self.s_gam))*jnp.log(descr3 + 1)
            descr = jnp.concatenate([descr, descr3],axis=-1)
        if self.level > 2: # meta-GGA
            if spin_scaling:
                descr4a = self.l_3(2*rho0_a, 4*gamma_a, 2*tau_a)
                descr4b = self.l_3(2*rho0_b, 4*gamma_b, 2*tau_b)
                descr4 = jnp.concatenate([jnp.expand_dims(descr4a,-1), jnp.expand_dims(descr4b,-1)],axis=-1)
                descr4 = descr4**3/(descr4**2+self.epsilon)
            else:
                descr4 = self.l_3(rho0_a + rho0_b, gamma_a + gamma_b + 2*gamma_ab, tau_a + tau_b)
                descr4 = 2*descr4/((1+zeta)**(5/3) + (1-zeta)**(5/3))
                descr4 = descr4**3/(descr4**2+self.epsilon)

                descr4 = jnp.expand_dims(descr4,-1)
            descr4 = jnp.log((descr4 + 1)/2)
            descr = jnp.concatenate([descr, descr4],axis=-1)
        if self.level > 3: # meta-GGA + V_estat
            # if spin_scaling:
            #     descr5a = self.l_4(2*rho0_a, 2*nl_a)
            #     descr5b = self.l_4(2*rho0_b, 2*nl_b)
            #     descr5 = jnp.log(jnp.stack([descr5a, descr5b],axis=-1) + self.loge)
            #     descr5 = descr5.view(descr5.size()[0],-1)
            # else:
            #     descr5= jnp.log(self.l_4(rho0_a + rho0_b, nl_a + nl_b) + self.loge)

            # descr = jnp.concatenate([descr, descr5],axis=-1)
            descr5 = self.nl_4(mf, dm).T
                
            descr = jnp.concatenate([descr, descr5], axis=-1)
        if spin_scaling and self.level <= 3:
            descr = jnp.transpose(jnp.reshape(descr,(jnp.shape(descr)[0],-1,2)), (2,0,1)) 
        return descr

    def eval_grid_models(self, rho, mf=None, dm=None):
        """
        eval_grid_models Evaluates all models stored in self.grid_models along with HEG exchange and correlation

        :param rho: List/array with [rho0_a,rho0_b,gamma_a,gamma_ab,gamma_b, dummy for laplacian, dummy for laplacian, tau_a, tau_b, non_loc_a, non_loc_b]
                    Shape assumes, for instance, that rho0_a = rho[:, 0], etc.
        :type rho: jax.Array
        :return: The exchange-correlation energy density (on the grid)
        :rtype: jax.Array
        """
        Exc = 0
        rho0_a = rho[:, 0]
        rho0_b = rho[:, 1]
        gamma_a = rho[:, 2]
        gamma_ab = rho[:, 3]
        gamma_b = rho[:, 4]
        tau_a = rho[:, 7]
        tau_b = rho[:, 8]
        nl = rho[:,9:]
        nl_size = jnp.size(nl, -1)//2
        nl_a = nl[:,:nl_size]
        nl_b = nl[:,nl_size:]

        C_F= 3/10*(3*np.pi**2)**(2/3)
        rho0_a_ueg = rho0_a
        rho0_b_ueg = rho0_b

        zeta = (rho0_a_ueg - rho0_b_ueg)/(rho0_a_ueg + rho0_b_ueg + 1e-8)
        rs = (4*np.pi/3*(rho0_a_ueg+rho0_b_ueg + 1e-8))**(-1/3)
        rs_a = (4*np.pi/3*(rho0_a_ueg + 1e-8))**(-1/3)
        rs_b = (4*np.pi/3*(rho0_b_ueg + 1e-8))**(-1/3)

        #initialize zero values for the ex/ec/grid values
        exc_a = jnp.zeros_like(rho0_a)
        exc_b = jnp.zeros_like(rho0_a)
        exc_ab = jnp.zeros_like(rho0_a)

        if self.debug:
            print('eval_grid_models nan summary:')
            print('zeta, rs, rs_a, rs_b, exc_a, exc_b, exc_ab')
            print('{}, {}, {}, {}, {}, {}, {}'.format(
                jnp.sum(jnp.any(jnp.isnan(zeta))),
                jnp.sum(jnp.any(jnp.isnan(rs))),
                jnp.sum(jnp.any(jnp.isnan(rs_a))),
                jnp.sum(jnp.any(jnp.isnan(rs_b))),
                jnp.sum(jnp.any(jnp.isnan(exc_a))),
                jnp.sum(jnp.any(jnp.isnan(exc_b))),
                jnp.sum(jnp.any(jnp.isnan(exc_ab))),                
            ))

        descr_dict = {}
        rho_tot = rho0_a + rho0_b
        #spin scaling false descriptors
        descr_dict[False] = self.get_descriptors(rho0_a, rho0_b, gamma_a, gamma_b,
                                                         gamma_ab, nl_a, nl_b, tau_a, tau_b, spin_scaling = False,
                                                mf = mf, dm = dm)
        #spin scaling true descriptors
        descr_dict[True] = self.get_descriptors(rho0_a, rho0_b, gamma_a, gamma_b,
                                                         gamma_ab, nl_a, nl_b, tau_a, tau_b, spin_scaling = True,
                                               mf = mf, dm = dm)

        def else_test_fun(exc, exc_b):
            if self.heg_mult:
                exc_b += (1 + exc[1])*self.heg_model(2*rho0_b_ueg)*(1-self.exx_a)
            else:
                exc_b += exc[1]*(1-self.exx_a)
            return exc_b
            
        def if_not_test_fun(exc, exc_b):
            exc_b += exc[0]*0
            return exc_b

        def gm_eval_func(grid_model, exc_a, exc_b, exc_ab):
            if not grid_model.spin_scaling:
                descr = descr_dict[False]
                print(f"spin_scaling = False; input descr to exc shape: {descr.shape}")
                exc = grid_model(descr)
                if jnp.ndim(exc) == 2: #If using spin decomposition
                    pw_alpha = self.pw_model(rs_a, jnp.ones_like(rs_a))
                    pw_beta = self.pw_model(rs_b, jnp.ones_like(rs_b))
                    pw = self.pw_model(rs, zeta)
                    ec_alpha = (1 + exc[:,0])*pw_alpha*rho0_a/(rho_tot+1e-8)
                    ec_beta =  (1 + exc[:,1])*pw_beta*rho0_b/(rho_tot+1e-8)
                    ec_mixed = (1 + exc[:,2])*(pw*rho_tot - pw_alpha*rho0_a - pw_beta*rho0_b)/(rho_tot+1e-8)
                    exc_ab += ec_alpha + ec_beta + ec_mixed
                else:
                    if self.pw_mult:
                        exc_ab += (1 + exc)*self.pw_model(rs, zeta)
                    else:
                        exc_ab += exc
            else:
                descr = descr_dict[True]
                print(f"spin_scaling = True; input descr to exc shape: {descr.shape}")
                exc = grid_model(descr)


                if self.heg_mult:
                    exc_a += (1 + exc[0])*self.heg_model(2*rho0_a_ueg)*(1-self.exx_a)
                else:
                    exc_a += exc[0]*(1-self.exx_a)
                test = jnp.sum(jnp.abs(rho0_b))
                exc_b = jax.lax.cond(test, else_test_fun, if_not_test_fun, exc, exc_b)

            return (exc_a, exc_b, exc_ab)

        if self.grid_models:
            exc_a, exc_b, exc_ab = gm_eval_func(self.grid_models[0], exc_a, exc_b, exc_ab)
            exc_a, exc_b, exc_ab = gm_eval_func(self.grid_models[1], exc_a, exc_b, exc_ab)                    
        else:
            if self.heg_mult:
                exc_a = self.heg_model(2*rho0_a_ueg)
                exc_b = self.heg_model(2*rho0_b_ueg)
            if self.pw_mult:
                exc_ab = self.pw_model(rs, zeta)


        exc = exc_a * (rho0_a_ueg/ (rho_tot + self.epsilon)) + exc_b*(rho0_b_ueg / (rho_tot + self.epsilon)) + exc_ab
        if self.debug:
            print('eval_grid_models nan summary:')
            print('zeta, rs, rs_a, rs_b, exc_a, exc_b, exc_ab')
            print('{}, {}, {}, {}, {}, {}, {}'.format(
                jnp.sum(jnp.any(jnp.isnan(zeta))),
                jnp.sum(jnp.any(jnp.isnan(rs))),
                jnp.sum(jnp.any(jnp.isnan(rs_a))),
                jnp.sum(jnp.any(jnp.isnan(rs_b))),
                jnp.sum(jnp.any(jnp.isnan(exc_a))),
                jnp.sum(jnp.any(jnp.isnan(exc_b))),
                jnp.sum(jnp.any(jnp.isnan(exc_ab))),                
            ))

        return jnp.expand_dims(exc, -1)

In [8]:
#update docs, only input =2 ??? for MGGA? holdover from sebastian for some reason
xnet = xce.net.eX(n_input = 2, use = [1, 2], ueg_limit=True, lob=1.174)
# I guess use default LOB
cnet = xce.net.eC(n_input = 4, use = [2, 3], ueg_limit=True)
blankxc = xce.xc.eXC(grid_models = [xnet, cnet], level=3)
p = '/home/awills/Documents/Research/xcquinox/models/pretrained/scan'
# xc = eqx.tree_deserialise_leaves(os.path.join(p, 'xc.eqx'), blankxc)
xc = blankxc

In [9]:
nlxnet = eX(n_input = 15, use = [], ueg_limit=True, lob=1.174)
nlcnet = eC(n_input = 13, use = [], ueg_limit=True)

nlxc = eXC(grid_models = [nlxnet, nlcnet], level=4)

In [81]:
class NL_E_loss(eqx.Module):
    def __init__(self):
        '''
        The standard energy loss module, RMSE loss of predicted vs. reference energies.
        '''
        super().__init__()

    def __call__(self, model, inp_dm, ref_en, ao_eval, grid_weights, mf):
        '''
        Computes the energy loss for a given model and associated input density matrix, atomic orbitals on the grid, and grid weights

        Loss is the RMSE energy, so predicted energy can potentially be a jax.Array of SCF guesses.

        :param model: The XC object whose forward pass predicts the XC energy based on the inputs here.
        :type model: xcquinox.xc.eXC
        :param inp_dm: The density matrix to pass into the network for density creation on the grid.
        :type inp_dm: jax.Array
        :param ref_en: The reference energy to take the loss with respect to.
        :type ref_en: jax.Array
        :param ao_eval: Atomic orbitals evaluated on the grid
        :type ao_eval: jax.Array
        :param grid_weights: pyscfad's grid weights for the reference calculation
        :type grid_weights: jax.Array
        :return: The RMSE error.
        :rtype: jax.Array
        '''
        e_pred = model(inp_dm, ao_eval, grid_weights, mf)
        eL = jnp.sqrt( jnp.mean((e_pred-ref_en)**2))
        return eL


In [10]:
cell = gtop.Cell()
a = 5.43
cell.atom = [['Si', [0,0,0]],
              ['Si', [a/4,a/4,a/4]]]
cell.a = jnp.asarray([[0, a/2, a/2],
                     [a/2, 0, a/2],
                     [a/2, a/2, 0]])
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build(trace_lattice_vectors=True)
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/pbc/gto/cell.py:243: UserWarning: Function cell.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/pbc/gto/cell.py:243: UserWarning: Function cell.dumps drops attribute a because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/pbc/gto/cell.py:243: UserWarning: Function cell.dumps drops attribute abc because it is not JSON-serializable
  warnings.warn(msg)


In [23]:
bands = mf.get_bands(kpts, cell=cell, kpts=kpts)

In [36]:
hcc = mf.get_hcore(cell, kpts)??

SyntaxError: invalid syntax (3855865464.py, line 1)

In [37]:
mf.get_hcore??

Signature: mf.get_hcore(cell=None, kpts=None)
Docstring:
Get the core Hamiltonian AO matrices at sampled k-points.

Args:
    kpts : (nkpts, 3) ndarray

Returns:
    hcore : (nkpts, nao, nao) ndarray
Source:   
def get_hcore(mf, cell=None, kpts=None):
    if cell is None:
        cell = mf.cell
    if kpts is None:
        kpts = mf.kpts
    if cell.pseudo:
        nuc = np.asarray(mf.with_df.get_pp(kpts))
    else:
        raise NotImplementedError
    if len(cell._ecpbas) > 0:
        raise NotImplementedError
    t = np.asarray(cell.pbc_intor('int1e_kin', comp=1, hermi=1, kpts=kpts))
    return nuc + t
File:      ~/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/pbc/scf/khf.py
Type:      method

In [40]:
tkpts = cell.pbc_intor('int1e_kin', comp=1, hermi=1, kpts=kpts)

In [41]:
tkpts.shape

(8, 8, 8)

In [45]:
mf.mo_coeff?

Type:        list
String form:
[array([[-5.11785471e-01+8.11619500e-19j, -9.18410647e-16-3.05577232e-17j,
           -8.54822546e-1 <...> -01+6.18995023e-02j,
           -2.74367362e-01-5.82588369e-01j, -7.07003474e-01+1.49945011e-15j]])]
Length:      8
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [44]:
mf.make_rdm1??

Signature: mf.make_rdm1(mo_coeff_kpts=None, mo_occ_kpts=None, **kwargs)
Docstring:
One-particle density matrix in AO representation

Args:
    mo_coeff : 2D ndarray
        Orbital coefficients. Each column is one orbital.
    mo_occ : 1D ndarray
        Occupancy
Returns:
    One-particle density matrix, 2D ndarray
Source:   
    def make_rdm1(self, mo_coeff_kpts=None, mo_occ_kpts=None, **kwargs):
        if mo_coeff_kpts is None:
            mo_coeff_kpts = self.mo_coeff
        if mo_occ_kpts is None:
            mo_occ_kpts = self.mo_occ
        return make_rdm1(mo_coeff_kpts, mo_occ_kpts, **kwargs)
File:      ~/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/pbc/scf/khf.py
Type:      method

In [29]:
type(mf)

pyscfad.pbc.scf.khf.KSCF

In [22]:
mf.get_bands??

Signature: mf.get_bands(kpts_band, cell=None, dm_kpts=None, kpts=None)
Source:   
    def get_bands(self, kpts_band, cell=None, dm_kpts=None, kpts=None):
        '''Get energy bands at the given (arbitrary) 'band' k-points.

        Returns:
            mo_energy : (nmo,) ndarray or a list of (nmo,) ndarray
                Bands energies E_n(k)
            mo_coeff : (nao, nmo) ndarray or a list of (nao,nmo) ndarray
                Band orbitals psi_n(k)
        '''
        if cell is None: cell = self.cell
        if dm_kpts is None: dm_kpts = self.make_rdm1()
        if kpts is None: kpts = self.kpts

        kpts_band = np.asarray(kpts_band)
        single_kpt_band = (kpts_band.ndim == 1)
        kpts_band = kpts_band.reshape(-1,3)

        fock = self.get_hcore(cell, kpts_band)
        fock = fock + self.get_veff(cell, dm_kpts, kpts=kpts, kpts_band=kpts_band)
        s1e = self.get_ovlp(cell, kpts_band)
        mo_energy, mo_coeff = self.eig(fock, s1e)
        if single_kpt_band:
 

In [24]:
kpts

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.30616192,  0.30616192, -0.30616192],
       [ 0.30616192, -0.30616192,  0.30616192],
       [ 0.61232384,  0.        ,  0.        ],
       [-0.30616192,  0.30616192,  0.30616192],
       [ 0.        ,  0.61232384,  0.        ],
       [ 0.        ,  0.        ,  0.61232384],
       [ 0.30616192,  0.30616192,  0.30616192]])

In [26]:
e_kn = bands[0]
vbmax = -99
for en in e_kn:
    vb_k = en[cell.nelectron//2-1]
    if vb_k > vbmax:
        vbmax = vb_k
e_kn = [en - vbmax for en in e_kn]

In [27]:
e_kn

[Array([-6.14187342e-01+0.j, -1.82144603e-11+0.j, -7.58227015e-12+0.j,
         0.00000000e+00+0.j,  5.50609333e-01+0.j,  5.50609333e-01+0.j,
         5.50609333e-01+0.j,  5.95700834e-01+0.j], dtype=complex128),
 Array([-0.49454436+0.j, -0.34280689+0.j, -0.07186577+0.j, -0.07186577+0.j,
         0.56397967+0.j,  0.66943433+0.j,  0.66943433+0.j,  0.93730272+0.j],      dtype=complex128),
 Array([-0.49454436+0.j, -0.34280689+0.j, -0.07186577+0.j, -0.07186577+0.j,
         0.56397967+0.j,  0.66943433+0.j,  0.66943433+0.j,  0.93730272+0.j],      dtype=complex128),
 Array([-0.39825575+0.j, -0.39825575+0.j, -0.15064707+0.j, -0.15064707+0.j,
         0.6127736 +0.j,  0.6127736 +0.j,  0.88688775+0.j,  0.88688775+0.j],      dtype=complex128),
 Array([-0.49454436+0.j, -0.34280689+0.j, -0.07186577+0.j, -0.07186577+0.j,
         0.56397967+0.j,  0.66943433+0.j,  0.66943433+0.j,  0.93730272+0.j],      dtype=complex128),
 Array([-0.39825575+0.j, -0.39825575+0.j, -0.15064707+0.j, -0.15064707+0.j,
    

In [28]:
bands[0]

[Array([-6.14566376e-01+0.j, -3.79034099e-04+0.j, -3.79034089e-04+0.j,
        -3.79034081e-04+0.j,  5.50230299e-01+0.j,  5.50230299e-01+0.j,
         5.50230299e-01+0.j,  5.95321800e-01+0.j], dtype=complex128),
 Array([-0.49492339+0.j, -0.34318592+0.j, -0.0722448 +0.j, -0.0722448 +0.j,
         0.56360064+0.j,  0.66905529+0.j,  0.66905529+0.j,  0.93692368+0.j],      dtype=complex128),
 Array([-0.49492339+0.j, -0.34318592+0.j, -0.0722448 +0.j, -0.0722448 +0.j,
         0.56360064+0.j,  0.66905529+0.j,  0.66905529+0.j,  0.93692368+0.j],      dtype=complex128),
 Array([-0.39863479+0.j, -0.39863479+0.j, -0.1510261 +0.j, -0.1510261 +0.j,
         0.61239457+0.j,  0.61239457+0.j,  0.88650872+0.j,  0.88650872+0.j],      dtype=complex128),
 Array([-0.49492339+0.j, -0.34318592+0.j, -0.0722448 +0.j, -0.0722448 +0.j,
         0.56360064+0.j,  0.66905529+0.j,  0.66905529+0.j,  0.93692368+0.j],      dtype=complex128),
 Array([-0.39863479+0.j, -0.39863479+0.j, -0.1510261 +0.j, -0.1510261 +0.j,
    

In [25]:
len(bands[0]), len(bands[1])

(8, 8)

In [91]:
nlxc(dm, ao_eval, mf.grids.weights, mf=mf)

PRIOR TO POST PROCESS 25274.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25274.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24114.0
MO VELE MAT 1127092224 23008.0
FINISHED POST PROCESS 23008.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25250.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25255.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24121.0
MO VELE MAT 1127092224 23000.0
FINISHED POST PROCESS 23000.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX.__call__, rho shape: (25728, 15)
spin_scaling = False; input descr to exc shape: (25728, 13)
eC.__call__

Array(-13.92249356, dtype=float64)

In [92]:
trainer = xce.train.xcTrainer(model=nlxc, optim=optax.adamw(1e-4), steps=10, loss = NL_E_loss(), do_jit=False)
newm = trainer(1, trainer.model, dms, energies, ao_evals, gws, mfs)


Epoch 0
Epoch 0 :: Batch 0/1
PRIOR TO POST PROCESS 25261.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25261.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24129.0
MO VELE MAT 1127092224 23012.0
FINISHED POST PROCESS 23012.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25267.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25270.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24137.0
MO VELE MAT 1127092224 23007.0
FINISHED POST PROCESS 23007.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX.__call__, rho shape: (25728, 15)
spin_scaling = False; input descr to exc s

Batch Loss = 0.2650463341350964
step=0, epoch_train_loss=0.2650463341350964
Epoch 1
Epoch 1 :: Batch 0/1
PRIOR TO POST PROCESS 25294.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25294.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24131.0
MO VELE MAT 1127092224 23003.0
FINISHED POST PROCESS 23003.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25271.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25271.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24146.0
MO VELE MAT 1127092224 23020.0
FINISHED POST PROCESS 23020.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX

Batch Loss = 0.2628795395406751
step=1, epoch_train_loss=0.2628795395406751
Epoch 2
Epoch 2 :: Batch 0/1
PRIOR TO POST PROCESS 25294.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25294.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24131.0
MO VELE MAT 1127092224 23001.0
FINISHED POST PROCESS 23001.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25269.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25269.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24126.0
MO VELE MAT 1127092224 23024.0
FINISHED POST PROCESS 23024.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX

Batch Loss = 0.26060511630729266
step=2, epoch_train_loss=0.26060511630729266
Epoch 3
Epoch 3 :: Batch 0/1
PRIOR TO POST PROCESS 25316.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25316.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24152.0
MO VELE MAT 1127092224 23016.0
FINISHED POST PROCESS 23016.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25278.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25278.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24155.0
MO VELE MAT 1127092224 23022.0
FINISHED POST PROCESS 23022.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)


Batch Loss = 0.2582163611488788
step=3, epoch_train_loss=0.2582163611488788
Epoch 4
Epoch 4 :: Batch 0/1
PRIOR TO POST PROCESS 25266.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25266.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24111.0
MO VELE MAT 1127092224 22992.0
FINISHED POST PROCESS 22992.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25268.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25268.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24127.0
MO VELE MAT 1127092224 22995.0
FINISHED POST PROCESS 22995.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX

Batch Loss = 0.2557060416681729
step=4, epoch_train_loss=0.2557060416681729
Epoch 5
Epoch 5 :: Batch 0/1
PRIOR TO POST PROCESS 25270.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25270.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24113.0
MO VELE MAT 1127092224 22988.0
FINISHED POST PROCESS 22988.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25240.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25239.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24122.0
MO VELE MAT 1127092224 23007.0
FINISHED POST PROCESS 23007.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX

Batch Loss = 0.25306635391473087
step=5, epoch_train_loss=0.25306635391473087
Epoch 6
Epoch 6 :: Batch 0/1
PRIOR TO POST PROCESS 25270.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25270.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24101.0
MO VELE MAT 1127092224 22977.0
FINISHED POST PROCESS 22977.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25238.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25244.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24105.0
MO VELE MAT 1127092224 22973.0
FINISHED POST PROCESS 22973.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)


Batch Loss = 0.25028887573473213
step=6, epoch_train_loss=0.25028887573473213
Epoch 7
Epoch 7 :: Batch 0/1
PRIOR TO POST PROCESS 25262.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25266.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24098.0
MO VELE MAT 1127092224 22985.0
FINISHED POST PROCESS 22985.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25241.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25241.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24115.0
MO VELE MAT 1127092224 22989.0
FINISHED POST PROCESS 22989.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)


Batch Loss = 0.2473645151592727
step=7, epoch_train_loss=0.2473645151592727
Epoch 8
Epoch 8 :: Batch 0/1
PRIOR TO POST PROCESS 25271.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25271.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24128.0
MO VELE MAT 1127092224 22980.0
FINISHED POST PROCESS 22980.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25244.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25244.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24121.0
MO VELE MAT 1127092224 22993.0
FINISHED POST PROCESS 22993.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX

Batch Loss = 0.24428345292365172
step=8, epoch_train_loss=0.24428345292365172
Epoch 9
Epoch 9 :: Batch 0/1
PRIOR TO POST PROCESS 25295.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25295.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24132.0
MO VELE MAT 1127092224 23009.0
FINISHED POST PROCESS 23009.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25272.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25275.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24141.0
MO VELE MAT 1127092224 23021.0
FINISHED POST PROCESS 23021.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)


In [93]:
en1 = nlxc(dms[0], ao_evals[0], gws[0], mfs[0])
en2 = newm(dms[0], ao_evals[0], gws[0], mfs[0])

PRIOR TO POST PROCESS 25220.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25220.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24072.0
MO VELE MAT 1127092224 22950.0
FINISHED POST PROCESS 22950.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
PRIOR TO POST PROCESS 25211.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 25211.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 24076.0
MO VELE MAT 1127092224 22938.0
FINISHED POST PROCESS 22938.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)
spin_scaling = True; input descr to exc shape: (25728, 15)
eX.__call__, rho shape: (25728, 15)
spin_scaling = False; input descr to exc shape: (25728, 13)
eC.__call__

In [94]:
abs(en1-energies[0]), abs(en2-energies[0])

(Array(0.26504633, dtype=float64), Array(0.23760792, dtype=float64))

In [ ]:
# ptscan = get_torch_xc(xctype='MGGA', pretrain_loc='/home/awills/Documents/Research/dpyscfl/models/pretrained/scan',
#                 nhidden=16)
# tgms = ptscan.grid_models
# t_x_w, t_x_b = get_torch_weights_and_biases(tgms[0].net)
# t_c_w, t_c_b = get_torch_weights_and_biases(tgms[1].net)

# xnet = init_linear_weight(xnet, seed=92017, new_weights = t_x_w, new_bias = t_x_b)
# cnet = init_linear_weight(cnet, seed=92017, new_weights = t_c_w, new_bias = t_c_b)
# gms = [xnet, cnet]
# xc = xce.xc.eXC(grid_models = gms, level=3)

In [ ]:
# xcd = get_torch_xc(xctype='MGGA', path='/home/awills/Documents/Research/torch_dpy/models/xcdiff/MODEL_MGGA/xc')
# tgms = xcd.grid_models
# t_x_w, t_x_b = get_torch_weights_and_biases(tgms[0].net)
# t_c_w, t_c_b = get_torch_weights_and_biases(tgms[1].net)

# xnet = init_linear_weight(xnet, seed=92017, new_weights = t_x_w, new_bias = t_x_b)
# cnet = init_linear_weight(cnet, seed=92017, new_weights = t_c_w, new_bias = t_c_b)
# gms = [xnet, cnet]
# xceqx = xce.xc.eXC(grid_models = gms, level=3)

In [ ]:
# xcdp = '/home/awills/Documents/Research/xcquinox/models/xcdiff'
# eqx.tree_serialise_leaves(os.path.join(xcdp, 'xc.eqx'), xceqx)

In [ ]:
# xc.grid_models[0].net.layers[0].weight - xceqx.grid_models[0].net.layers[0].weight

In [ ]:
# p = '/home/awills/Documents/Research/xcquinox/models/pretrained/scan'
# eqx.tree_serialise_leaves(os.path.join(p, 'xc.eqx'), xc)

Test molecule with pyscfad

In [15]:
trainms = read('/home/awills/Documents/Research/torch_dpy/subset09_nf/subat_ref_corrected.traj', ':')
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []
for idx, at in enumerate(trainms[1:2]):
    name, mol = xce.utils.ase_atoms_to_mol(at, basis='def2tzvpd')
    mol.build()
    mols.append(mol)
    mf = dft.RKS(mol, xc='SCAN')
    e_tot = mf.kernel()
    mfs.append(mf)
    dm = mf.make_rdm1()
    ao_eval = jnp.array(mf._numint.eval_ao(mol, mf.grids.coords, deriv=2))
    energies.append(mf.get_veff().exc)
    dms.append(dm)
    ogds.append(dm.shape)
    ao_evals.append(ao_eval)
    gws.append(mf.grids.weights)
    ts.append(mol.intor('int1e_kin'))
    vs.append(mol.intor('int1e_nuc'))
    mo_occs.append(mf.mo_occ)
    hcs.append(mf.get_hcore())
    eris.append(mol.intor('int2e'))
    ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp'))))
    hologaps.append(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1])

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -109.525964831721


In [22]:
xc.grid_models[0].spin_scaling

True

In [24]:
xc(dm, ao_eval, mf.grids.weights), mf.get_veff().exc

(Array(-12.76261576, dtype=float64), -13.657447221972637)

In [25]:
an = RKSAnalyzer(calc=mf)
#these are the OUTPUTs from contract_exchange_descriptors, which ARE the rotationally invariant quantities.
#per the final call in _get_x_helper
#the return quantities are indexed as
# res
# 0:  rho
# 1:  s
# 2:  alpha
# 3:  g0
# 4:  norm(g1)**2
# 5:  g1 dot svec
# 6:  norm(g2)**2
# 7:  svec dot g2 dot svec
# 8:  g1 dot g2 dot svec
# 9:  g1 dot g2 dot g1
# 10: g0-r^2
# 11: g0-r^4
descr = get_exchange_descriptors2(an, restricted=True, version='c', auxbasis='def2tzvp')

PRIOR TO POST PROCESS 27858.0
post_process, self.grid shape: (25728, 3)
NUMBER OF CHUNKS RHF 1 float64 27830.0
get_vele_mat, returned shape: (74, 74, 25728)
get_vele_mat, points shape: (25728, 3)
AO VELE MAT 1127092224 (25728, 74, 74)
MEM NOW 26674.0
MO VELE MAT 1127092224 25510.0
FINISHED POST PROCESS 25510.0
nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)


In [26]:
descr.shape

(12, 25728)

In [27]:
vgf = lambda x: xc(x, ao_evals[0], gws[0])
dmp, moe, moc = xce.utils.get_dm_moe(dms[0], eris[0], vgf, mo_occs[0], hcs[0],ss[0], ogds[0])

[74] (74,)
(74, 74) (74, 74)
Spin unpolarized make_rdm1()


In [28]:
descr3 = jnp.asarray(get_exchange_descriptors2(an, restricted=True, version='c', auxbasis='def2tzvp',
                                   rdm1=True, dm=np.asarray(dmp), inmol=True, mol=mol, ingrid=True, grid=mf.grids))
                                   

nao in get_exchange_descriptors2: 74
naux in get_exchange_descriptors2: 74
ao_to_aux shape: (74, 74, 74)


In [30]:
nlstart_i, nlend_i = (3,12)
nldesc = descr3[nlstart_i:nlend_i]

In [31]:
nldesc.shape

(9, 25728)

Create silicon cell

In [ ]:
cell = gtop.Cell()
a = 5.43
cell.atom = [['Si', [0,0,0]],
              ['Si', [a/4,a/4,a/4]]]
cell.a = jnp.asarray([[0, a/2, a/2],
                     [a/2, 0, a/2],
                     [a/2, a/2, 0]])
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build(trace_lattice_vectors=True)
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()